# 06 compare the different FDG runs / reference regions

versions needed to compare:

Data versions:
original reference region + original subject set
Paracentral-gm reference region + original subject set
Paracentral-gm reference region + new subject set
RAS-ref1 reference region + original subject set
RAS-ref1 reference region + new subject set



age correction of original data is for plasma data only, linear regression using all_data[all_data['source'].isin(['Control']) & all_data['Subject_Label'].isin(['AANDS', 'ADRC'])])

In [1]:
# install required packages - commented out so it doesn't install every time
#%conda install -n Lauren openpyxl numpy pandas statsmodels plotnine matplotlib scikit-learn scipy mizani nbconvert pandoc pyreadstat kmodes seaborn

# import required packages
import numpy as np;
import pandas as pd;
from sklearn import preprocessing 
import statsmodels as sm;
import statsmodels.formula.api as smf;
import plotnine as p9;
import pickle



# read in and clean data

In [2]:
data_folder_loc = "//admsyn/Primary/ADM/CustomerStudies/Rockefeller/Riluzole_Biomarkers/"
code_folder_loc = "//admsyn/homes/@DH-ADMDX/0/lauren.koenig-1606/code/Riluzole FDG/"

In [3]:
# read in data
results_composite, temp, data1 = pd.read_pickle(code_folder_loc + '/output/04_data_composite_rr.pkl')
results_pons, temp, data2 = pd.read_pickle(code_folder_loc + '/output/04_data_pons_rr.pkl')
results_para, temp, data3 = pd.read_pickle(code_folder_loc + '/output/04_data_para_rr.pkl')
results_new_para, temp, data4 = pd.read_pickle(code_folder_loc + '/output/04_data_new_para_rr.pkl')
results_ras, temp, data5 = pd.read_pickle(code_folder_loc + '/output/04_data_ras_rr.pkl')
results_subset_new_para, temp, data5_subset = pd.read_pickle(code_folder_loc + '/output/04_data_new_para_rr_subset.pkl')


results_orig_new_para, temp, data6 = pd.read_pickle(code_folder_loc + '/output/with_RIL8_included/04_data_new_para_rr.pkl')


In [4]:
rename_columns = ['cor_p_baseline', 'cor_r_baseline', 'cor_n',
       'mv_age_p_baseline', 'mv_age_B_baseline', 'mv_age_n',
       'mv_ageedu_p_baseline', 'mv_ageedu_B_baseline', 'mv_ageedu_n',
       'mv_aeasr_p_baseline', 'mv_aeasr_B_baseline', 'mv_aeasr_n']

In [5]:
results_composite = results_composite.rename(columns = dict(zip(rename_columns, 
['composite_' + ele for ele in rename_columns])))

results_pons = results_pons.rename(columns = dict(zip(rename_columns, 
['pons_' + ele for ele in rename_columns])))

results_para = results_para.rename(columns = dict(zip(rename_columns, 
['para_' + ele for ele in rename_columns])))

results_new_para = results_new_para.rename(columns = dict(zip(rename_columns, 
['new_para_' + ele for ele in rename_columns])))

results_ras = results_ras.rename(columns = dict(zip(rename_columns, 
['ras_' + ele for ele in rename_columns])))

results_subset_new_para = results_subset_new_para.rename(columns = dict(zip(rename_columns, 
['subset_new_para_' + ele for ele in rename_columns])))

results_orig_new_para = results_orig_new_para.rename(columns = dict(zip(rename_columns, 
['orig_new_para_' + ele for ele in rename_columns])))


In [6]:
# define biomarker groups
FDG_columns = ['Avg_MedOrbFrontal', 'Graycer_gm', 'Avg_PCC', 'Avg_Hip', 'MTL_gm', 'CO', 'Temp', 'SensMot_gm', 'L_Hip',
 'Vermis_gm', 'FRONTAL_gm', 'AC_gm', 'Precun_gm', 'Par_gm', 'RASref1_gm', 'Temp_gm', 'Paracentral_gm', 'R_Hip', 'PostCing_gm']

plasma_columns = [ 'Ab40', 'Ab42', 'Ab42_40', 'GFAP','NFL',  'pTau181', 'pTau217',  'pTau231', 'pTau181_Ab42']
log10_plasma_columns = [s + '_log10' for s in plasma_columns]

cog_columns = ['MMSE', 'adascogtotal', 'bvrt', 'dstotal','tma','tmb',  'cowattotal', 'cdrtotal', 'cdrsum', 'adltotal', 'npitotal', 'gds']
vol_columns = ['VOL_Precun_Lz', 'VOL_Precun_Rz', 'VOL_InfPar_Lz', 'VOL_InfPar_Rz', 'VOL_Hip_Lz', 'VOL_Hip_Rz', 'VOL_TotalGrayz', 'VOL_Inf_Mid_Fus_Temp_Lz', 
'VOL_Inf_Mid_Fus_Temp_Rz','VOL_MidFront_Lz', 'VOL_MidFront_Rz']


# Composite vs New Para

In [7]:
comp_new_para = pd.merge(results_composite, results_new_para, how = 'inner')
comp_new_para

,var1,var2,composite_cor_p_baseline,composite_cor_r_baseline,composite_cor_n,composite_mv_age_p_baseline,composite_mv_age_B_baseline,composite_mv_age_n,composite_mv_ageedu_p_baseline,composite_mv_ageedu_B_baseline,...,new_para_cor_n,new_para_mv_age_p_baseline,new_para_mv_age_B_baseline,new_para_mv_age_n,new_para_mv_ageedu_p_baseline,new_para_mv_ageedu_B_baseline,new_para_mv_ageedu_n,new_para_mv_aeasr_p_baseline,new_para_mv_aeasr_B_baseline,new_para_mv_aeasr_n
0,Ab40,MTL_gm,0.248673,-0.200280,35,0.878973,-0.029233,35,0.750044,-0.071179,...,40,0.888369,-0.024268,39,0.682126,-0.081689,39,0.998305,0.000562,34
1,Ab40,FRONTAL_gm,0.721466,-0.062469,35,0.592292,0.094922,35,0.643444,0.090122,...,40,0.480051,0.118418,39,0.568897,0.104043,39,0.529761,0.139394,34
2,Ab40,AC_gm,0.725668,-0.061489,35,0.991560,0.001962,35,0.940486,-0.014797,...,40,0.768005,-0.051412,39,0.672515,-0.077033,39,0.941134,0.017332,34
3,Ab40,Precun_gm,0.914754,-0.018774,35,0.910323,-0.020120,35,0.824361,-0.043250,...,40,0.940730,0.012692,39,0.905003,-0.022502,39,0.899370,-0.027883,34
4,Ab40,Par_gm,0.957331,0.009384,35,0.793883,0.048686,35,0.846295,0.038114,...,40,0.847880,0.033841,39,0.920992,0.018078,39,0.734919,0.068926,34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
931,pTau181_Ab42,VOL_Precun_InfPar_Supramarg_Rz,0.511106,0.124805,30,0.108750,0.311451,30,0.001254,0.563194,...,30,0.108750,0.311451,30,0.001254,0.563194,30,0.003669,0.630301,25
932,pTau181_Ab42,VOL_LatOccLingCun_Lz,0.203086,0.239155,30,0.071093,0.331401,30,0.026065,0.362568,...,30,0.071093,0.331401,30,0.026065,0.362568,30,0.007344,0.512186,25
933,pTau181_Ab42,VOL_LatOccLingCun_Rz,0.117861,0.291657,30,0.075020,0.327445,30,0.009236,0.423286,...,30,0.075020,0.327445,30,0.009236,0.423286,30,0.009140,0.503818,25
934,pTau181_Ab42,VOL_InfParSupra_Lz,0.556047,0.111902,30,0.144645,0.306248,30,0.036780,0.390568,...,30,0.144645,0.306248,30,0.036780,0.390568,30,0.037092,0.463150,25


## cor

In [8]:
comp_new_para[(comp_new_para['composite_cor_p_baseline'] < 0.05) & (comp_new_para['new_para_cor_p_baseline'] >= 0.05)]

,var1,var2,composite_cor_p_baseline,composite_cor_r_baseline,composite_cor_n,composite_mv_age_p_baseline,composite_mv_age_B_baseline,composite_mv_age_n,composite_mv_ageedu_p_baseline,composite_mv_ageedu_B_baseline,...,new_para_cor_n,new_para_mv_age_p_baseline,new_para_mv_age_B_baseline,new_para_mv_age_n,new_para_mv_ageedu_p_baseline,new_para_mv_ageedu_B_baseline,new_para_mv_ageedu_n,new_para_mv_aeasr_p_baseline,new_para_mv_aeasr_B_baseline,new_para_mv_aeasr_n


In [9]:
comp_new_para[(comp_new_para['composite_cor_p_baseline'] >= 0.05) & (comp_new_para['new_para_cor_p_baseline'] < 0.05)]

,var1,var2,composite_cor_p_baseline,composite_cor_r_baseline,composite_cor_n,composite_mv_age_p_baseline,composite_mv_age_B_baseline,composite_mv_age_n,composite_mv_ageedu_p_baseline,composite_mv_ageedu_B_baseline,...,new_para_cor_n,new_para_mv_age_p_baseline,new_para_mv_age_B_baseline,new_para_mv_age_n,new_para_mv_ageedu_p_baseline,new_para_mv_ageedu_B_baseline,new_para_mv_ageedu_n,new_para_mv_aeasr_p_baseline,new_para_mv_aeasr_B_baseline,new_para_mv_aeasr_n
19,Ab42_40,Temp_gm,0.085381,0.314006,31,0.151228,0.265333,31,0.165658,0.271941,...,36,0.111153,0.268312,35,0.111653,0.283013,35,0.101392,0.359751,30


In [10]:
(comp_new_para['composite_cor_p_baseline']  - comp_new_para['new_para_cor_p_baseline']).describe().round(3)

count    936.000
mean       0.000
std        0.037
min       -0.374
25%        0.000
50%        0.000
75%        0.000
max        0.302
dtype: float64

## age

In [11]:
comp_new_para.loc[(comp_new_para['composite_mv_age_p_baseline'] < 0.05) & (comp_new_para['new_para_mv_age_p_baseline'] >= 0.05), ['var1', 'var2', 'composite_mv_age_p_baseline',
       'composite_mv_age_B_baseline', 'composite_mv_age_n', 'new_para_mv_age_p_baseline', 'new_para_mv_age_B_baseline',
       'new_para_mv_age_n']]

,var1,var2,composite_mv_age_p_baseline,composite_mv_age_B_baseline,composite_mv_age_n,new_para_mv_age_p_baseline,new_para_mv_age_B_baseline,new_para_mv_age_n
25,GFAP,Par_gm,0.040060,-0.357448,37,0.073024,-0.302208,41
41,pTau181,PostCing_gm,0.034887,0.349089,37,0.052931,0.304133,41


In [12]:
comp_new_para[(comp_new_para['composite_mv_age_p_baseline'] >= 0.05) & (comp_new_para['new_para_mv_age_p_baseline'] < 0.05)]

,var1,var2,composite_cor_p_baseline,composite_cor_r_baseline,composite_cor_n,composite_mv_age_p_baseline,composite_mv_age_B_baseline,composite_mv_age_n,composite_mv_ageedu_p_baseline,composite_mv_ageedu_B_baseline,...,new_para_cor_n,new_para_mv_age_p_baseline,new_para_mv_age_B_baseline,new_para_mv_age_n,new_para_mv_ageedu_p_baseline,new_para_mv_ageedu_B_baseline,new_para_mv_ageedu_n,new_para_mv_aeasr_p_baseline,new_para_mv_aeasr_B_baseline,new_para_mv_aeasr_n
22,GFAP,FRONTAL_gm,0.081167,-0.290456,37,0.054203,-0.320741,37,0.095514,-0.299854,...,42,0.038012,-0.331277,41,0.116266,-0.263333,41,0.051380,-0.361989,36
24,GFAP,Precun_gm,0.273629,-0.184761,37,0.078996,-0.295709,37,0.135376,-0.270451,...,42,0.042803,-0.328679,41,0.141319,-0.254570,41,0.115938,-0.297232,36


In [13]:
(comp_new_para['composite_mv_age_p_baseline']  - comp_new_para['new_para_mv_age_p_baseline'])[(comp_new_para['composite_mv_age_p_baseline']  - comp_new_para['new_para_mv_age_p_baseline']) != 0].describe().round(3).round(2)

count    541.00
mean       0.00
std        0.04
min       -0.39
25%       -0.00
50%       -0.00
75%        0.00
max        0.36
dtype: float64

In [14]:
(comp_new_para['composite_mv_age_p_baseline']  - comp_new_para['new_para_mv_age_p_baseline']).describe().round(3)

count    936.000
mean       0.000
std        0.032
min       -0.392
25%       -0.000
50%        0.000
75%        0.000
max        0.358
dtype: float64

In [15]:
(comp_new_para['composite_mv_age_n']  - comp_new_para['new_para_mv_age_n']).value_counts()

 0    873
-4     56
-3      7
Name: count, dtype: int64

## age education

In [16]:
comp_new_para.loc[(comp_new_para['composite_mv_ageedu_p_baseline'] < 0.05) & (comp_new_para['new_para_mv_ageedu_p_baseline'] >= 0.05), ['var1', 'var2', 'composite_mv_ageedu_p_baseline',
       'composite_mv_ageedu_B_baseline', 'composite_mv_ageedu_n', 'new_para_mv_ageedu_p_baseline', 'new_para_mv_ageedu_B_baseline',
       'new_para_mv_ageedu_n']]

,var1,var2,composite_mv_ageedu_p_baseline,composite_mv_ageedu_B_baseline,composite_mv_ageedu_n,new_para_mv_ageedu_p_baseline,new_para_mv_ageedu_B_baseline,new_para_mv_ageedu_n


In [17]:
comp_new_para.loc[(comp_new_para['composite_mv_ageedu_p_baseline'] >= 0.05) & (comp_new_para['new_para_mv_ageedu_p_baseline'] < 0.05), ['var1', 'var2', 'composite_mv_ageedu_p_baseline',
       'composite_mv_ageedu_B_baseline', 'composite_mv_ageedu_n', 'new_para_mv_ageedu_p_baseline', 'new_para_mv_ageedu_B_baseline',
       'new_para_mv_ageedu_n']]

,var1,var2,composite_mv_ageedu_p_baseline,composite_mv_ageedu_B_baseline,composite_mv_ageedu_n,new_para_mv_ageedu_p_baseline,new_para_mv_ageedu_B_baseline,new_para_mv_ageedu_n
36,pTau181,FRONTAL_gm,0.07369,0.314175,37,0.031049,0.348622,41


In [18]:
(comp_new_para['composite_mv_ageedu_p_baseline']  - comp_new_para['new_para_mv_ageedu_p_baseline']).describe().round(3)

count    936.000
mean       0.002
std        0.034
min       -0.333
25%       -0.000
50%        0.000
75%        0.000
max        0.371
dtype: float64

In [19]:
comp_new_para.loc[(abs(comp_new_para['composite_mv_ageedu_p_baseline']  - comp_new_para['new_para_mv_ageedu_p_baseline'])> 0.1), ['var1', 'var2', 'composite_mv_ageedu_p_baseline',
       'composite_mv_ageedu_B_baseline', 'composite_mv_ageedu_n', 'new_para_mv_ageedu_p_baseline', 'new_para_mv_ageedu_B_baseline',
       'new_para_mv_ageedu_n']].sort_values('new_para_mv_ageedu_p_baseline')

,var1,var2,composite_mv_ageedu_p_baseline,composite_mv_ageedu_B_baseline,composite_mv_ageedu_n,new_para_mv_ageedu_p_baseline,new_para_mv_ageedu_B_baseline,new_para_mv_ageedu_n
40,pTau181,Temp_gm,0.355238,0.160642,37,0.109402,0.255602,41
54,pTau231,Temp_gm,0.281571,0.205149,33,0.158122,0.249653,37
38,pTau181,Precun_gm,0.320860,0.177825,37,0.193241,0.221054,41
8,Ab42,FRONTAL_gm,0.457171,0.155965,31,0.293305,0.194438,35
53,pTau231,Par_gm,0.503346,0.141283,33,0.350825,0.179169,37
51,pTau231,AC_gm,0.495923,0.137379,33,0.386835,0.159167,37
42,pTau217,MTL_gm,0.320285,-0.206628,34,0.420969,-0.147778,37
30,NFL,AC_gm,0.781847,-0.051425,37,0.495065,-0.117272,41
27,GFAP,PostCing_gm,0.741130,0.059317,37,0.507940,0.111180,41
61,pTau181_Ab42,Temp_gm,0.879715,-0.028867,31,0.509179,0.111678,35


## age education apoe sex race

In [20]:
comp_new_para.columns

Index(['var1', 'var2', 'composite_cor_p_baseline', 'composite_cor_r_baseline',
       'composite_cor_n', 'composite_mv_age_p_baseline',
       'composite_mv_age_B_baseline', 'composite_mv_age_n',
       'composite_mv_ageedu_p_baseline', 'composite_mv_ageedu_B_baseline',
       'composite_mv_ageedu_n', 'composite_mv_aeasr_p_baseline',
       'composite_mv_aeasr_B_baseline', 'composite_mv_aeasr_n', 'type',
       'new_para_cor_p_baseline', 'new_para_cor_r_baseline', 'new_para_cor_n',
       'new_para_mv_age_p_baseline', 'new_para_mv_age_B_baseline',
       'new_para_mv_age_n', 'new_para_mv_ageedu_p_baseline',
       'new_para_mv_ageedu_B_baseline', 'new_para_mv_ageedu_n',
       'new_para_mv_aeasr_p_baseline', 'new_para_mv_aeasr_B_baseline',
       'new_para_mv_aeasr_n'],
      dtype='object')

In [21]:
comp_new_para[(comp_new_para['composite_mv_aeasr_p_baseline'] < 0.05) & (comp_new_para['new_para_mv_aeasr_p_baseline'] >= 0.05)]

,var1,var2,composite_cor_p_baseline,composite_cor_r_baseline,composite_cor_n,composite_mv_age_p_baseline,composite_mv_age_B_baseline,composite_mv_age_n,composite_mv_ageedu_p_baseline,composite_mv_ageedu_B_baseline,...,new_para_cor_n,new_para_mv_age_p_baseline,new_para_mv_age_B_baseline,new_para_mv_age_n,new_para_mv_ageedu_p_baseline,new_para_mv_ageedu_B_baseline,new_para_mv_ageedu_n,new_para_mv_aeasr_p_baseline,new_para_mv_aeasr_B_baseline,new_para_mv_aeasr_n


In [22]:
comp_new_para[(comp_new_para['composite_mv_aeasr_p_baseline'] >= 0.05) & (comp_new_para['new_para_mv_aeasr_p_baseline'] < 0.05)]

,var1,var2,composite_cor_p_baseline,composite_cor_r_baseline,composite_cor_n,composite_mv_age_p_baseline,composite_mv_age_B_baseline,composite_mv_age_n,composite_mv_ageedu_p_baseline,composite_mv_ageedu_B_baseline,...,new_para_cor_n,new_para_mv_age_p_baseline,new_para_mv_age_B_baseline,new_para_mv_age_n,new_para_mv_ageedu_p_baseline,new_para_mv_ageedu_B_baseline,new_para_mv_ageedu_n,new_para_mv_aeasr_p_baseline,new_para_mv_aeasr_B_baseline,new_para_mv_aeasr_n


In [23]:
(comp_new_para['composite_mv_aeasr_p_baseline']  - comp_new_para['new_para_mv_aeasr_p_baseline']).describe().round(3)

count    936.000
mean      -0.000
std        0.001
min       -0.007
25%        0.000
50%        0.000
75%        0.000
max        0.003
dtype: float64

# new_para vs subset new_para

In [24]:
new_para_new_para = pd.merge(results_new_para, results_subset_new_para, how = 'inner')
new_para_new_para

,var1,var2,new_para_cor_p_baseline,new_para_cor_r_baseline,new_para_cor_n,new_para_mv_age_p_baseline,new_para_mv_age_B_baseline,new_para_mv_age_n,new_para_mv_ageedu_p_baseline,new_para_mv_ageedu_B_baseline,...,subset_new_para_cor_n,subset_new_para_mv_age_p_baseline,subset_new_para_mv_age_B_baseline,subset_new_para_mv_age_n,subset_new_para_mv_ageedu_p_baseline,subset_new_para_mv_ageedu_B_baseline,subset_new_para_mv_ageedu_n,subset_new_para_mv_aeasr_p_baseline,subset_new_para_mv_aeasr_B_baseline,subset_new_para_mv_aeasr_n
0,Ab40,MTL_gm,0.280507,-0.174859,40,0.888369,-0.024268,39,0.682126,-0.081689,...,35,0.878462,-0.029371,35,0.749256,-0.071472,35,0.998305,0.000534,34
1,Ab40,SensMot_gm,0.061871,0.297936,40,0.068632,0.299515,39,0.076043,0.295868,...,35,0.031817,0.370376,35,0.036041,0.371019,35,0.025934,0.413377,34
2,Ab40,FRONTAL_gm,0.921283,-0.016135,40,0.480051,0.118418,39,0.568897,0.104043,...,35,0.592202,0.094937,35,0.643271,0.090108,35,0.529761,0.134773,34
3,Ab40,AC_gm,0.466170,-0.118574,40,0.768005,-0.051412,39,0.672515,-0.077033,...,35,0.990301,0.002255,35,0.941846,-0.014461,35,0.941134,0.017031,34
4,Ab40,Precun_gm,0.706867,0.061351,40,0.940730,0.012692,39,0.905003,-0.022502,...,35,0.906638,-0.020953,35,0.820724,-0.044154,35,0.899370,-0.027032,34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
958,pTau181_Ab42,VOL_Precun_InfPar_Supramarg_Rz,0.511106,0.124805,30,0.108750,0.311451,30,0.001254,0.563194,...,26,0.118392,0.318164,26,0.004857,0.508249,26,0.003669,0.611029,25
959,pTau181_Ab42,VOL_LatOccLingCun_Lz,0.203086,0.239155,30,0.071093,0.331401,30,0.026065,0.362568,...,26,0.012973,0.481605,26,0.005011,0.478398,26,0.007344,0.540949,25
960,pTau181_Ab42,VOL_LatOccLingCun_Rz,0.117861,0.291657,30,0.075020,0.327445,30,0.009236,0.423286,...,26,0.014186,0.484985,26,0.005585,0.482153,26,0.009140,0.530937,25
961,pTau181_Ab42,VOL_InfParSupra_Lz,0.556047,0.111902,30,0.144645,0.306248,30,0.036780,0.390568,...,26,0.174088,0.302604,26,0.037331,0.411717,26,0.037092,0.474860,25


## cor

In [25]:
new_para_new_para[(new_para_new_para['new_para_cor_p_baseline'] < 0.05) & (new_para_new_para['subset_new_para_cor_p_baseline'] >= 0.05)]

,var1,var2,new_para_cor_p_baseline,new_para_cor_r_baseline,new_para_cor_n,new_para_mv_age_p_baseline,new_para_mv_age_B_baseline,new_para_mv_age_n,new_para_mv_ageedu_p_baseline,new_para_mv_ageedu_B_baseline,...,subset_new_para_cor_n,subset_new_para_mv_age_p_baseline,subset_new_para_mv_age_B_baseline,subset_new_para_mv_age_n,subset_new_para_mv_ageedu_p_baseline,subset_new_para_mv_ageedu_B_baseline,subset_new_para_mv_ageedu_n,subset_new_para_mv_aeasr_p_baseline,subset_new_para_mv_aeasr_B_baseline,subset_new_para_mv_aeasr_n
27,Ab42_40,Temp_gm,0.045535,0.335393,36,0.111153,0.268312,35,0.111653,0.283013,...,31,0.151501,0.265198,31,0.165949,0.271814,31,0.101392,0.328189,30
94,Ab40,pTau181,0.046590,-0.308833,42,0.141322,-0.249397,39,0.167331,-0.244977,...,37,0.228867,-0.214153,35,0.247990,-0.213516,35,0.301894,-0.199417,34
104,Ab42,pTau231,0.047037,-0.333226,36,0.099934,-0.289168,33,0.167995,-0.245285,...,31,0.192555,-0.255785,29,0.271679,-0.215629,29,0.346202,-0.204612,28
130,pTau181,Ab40,0.046590,-0.308833,42,0.141322,-0.237027,39,0.167331,-0.219442,...,37,0.228867,-0.209738,35,0.247990,-0.200479,35,0.301894,-0.197635,34
147,pTau231,Ab42,0.047037,-0.333226,36,0.099934,-0.303087,33,0.167995,-0.262966,...,31,0.192555,-0.251771,29,0.271679,-0.223097,29,0.346202,-0.206953,28
295,Ab40,VOL_OrbitFront_Rz,0.030852,-0.370817,34,0.245215,-0.207011,34,0.246113,-0.209020,...,30,0.387048,-0.166385,30,0.393923,-0.166669,30,0.579455,-0.122464,29
296,Ab40,VOL_Insula_Lz,0.036945,-0.359206,34,0.168095,-0.244621,34,0.168220,-0.247385,...,30,0.223834,-0.231499,30,0.213148,-0.241785,30,0.189327,-0.270559,29
354,Ab42,VOL_Fusi_Rz,0.049328,-0.361998,30,0.054651,-0.354205,30,0.026585,-0.412502,...,26,0.091279,-0.348408,26,0.072035,-0.371034,26,0.036197,-0.471892,25
360,Ab42,VOL_SupTemp_Rz,0.049328,-0.361998,30,0.035937,-0.383221,30,0.025722,-0.406561,...,26,0.062611,-0.370157,26,0.041274,-0.406264,26,0.007725,-0.578155,25
371,Ab42,VOL_OrbitFront_Lz,0.027636,-0.402047,30,0.019698,-0.422847,30,0.029747,-0.401800,...,26,0.023761,-0.443416,26,0.021743,-0.448792,26,0.004499,-0.602492,25


In [26]:
new_para_new_para[(new_para_new_para['new_para_cor_p_baseline'] >= 0.05) & (new_para_new_para['subset_new_para_cor_p_baseline'] < 0.05)]

,var1,var2,new_para_cor_p_baseline,new_para_cor_r_baseline,new_para_cor_n,new_para_mv_age_p_baseline,new_para_mv_age_B_baseline,new_para_mv_age_n,new_para_mv_ageedu_p_baseline,new_para_mv_ageedu_B_baseline,...,subset_new_para_cor_n,subset_new_para_mv_age_p_baseline,subset_new_para_mv_age_B_baseline,subset_new_para_mv_age_n,subset_new_para_mv_ageedu_p_baseline,subset_new_para_mv_ageedu_B_baseline,subset_new_para_mv_ageedu_n,subset_new_para_mv_aeasr_p_baseline,subset_new_para_mv_aeasr_B_baseline,subset_new_para_mv_aeasr_n
1,Ab40,SensMot_gm,0.061871,0.297936,40,0.068632,0.299515,39,0.076043,0.295868,...,35,0.031817,0.370376,35,0.036041,0.371019,35,0.025934,0.413377,34
11,Ab42,SensMot_gm,0.057967,0.318939,36,0.099368,0.276843,35,0.088305,0.284968,...,31,0.044969,0.370545,31,0.049590,0.362814,31,0.082676,0.367633,30
81,pTau181_Ab42,SensMot_gm,0.174979,-0.231145,36,0.485982,-0.119258,35,0.416484,-0.132812,...,31,0.054755,-0.356812,31,0.055637,-0.344153,31,0.050452,-0.401771,30
280,Ab40,VOL_MidTemp_Lz,0.100089,-0.286784,34,0.075045,-0.328548,34,0.033639,-0.419898,...,30,0.059569,-0.363754,30,0.025010,-0.473991,30,0.074042,-0.426408,29
331,Ab40,VOL_InfMidTemp_Lz,0.094183,-0.291673,34,0.092559,-0.309208,34,0.056812,-0.366006,...,30,0.085648,-0.329580,30,0.063834,-0.370180,30,0.112001,-0.346371,29
337,Ab40,VOL_Inf_Mid_Fus_Temp_Lz,0.058722,-0.327426,34,0.056431,-0.347208,34,0.039202,-0.386115,...,30,0.068366,-0.348004,30,0.055825,-0.376350,30,0.083807,-0.380138,29
376,Ab42,VOL_InfFront_Rz,0.081625,-0.323062,30,0.072545,-0.334214,30,0.073285,-0.332940,...,26,0.087897,-0.343744,26,0.086324,-0.345238,26,0.062612,-0.410092,25
414,Ab42,VOL_Inf_Mid_Fus_Temp_Lz,0.054153,-0.355101,30,0.014190,-0.449607,30,0.004061,-0.531441,...,26,0.015591,-0.469492,26,0.005960,-0.534637,26,0.000870,-0.693129,25
506,GFAP,VOL_ParaHip_Rz,0.191293,0.222909,36,0.148370,0.249701,36,0.059296,0.327759,...,32,0.022871,0.402493,32,0.016516,0.435375,32,0.021614,0.433801,31
623,NFL,VOL_AntCingulate_Rz,0.143815,0.248552,36,0.144111,0.246872,36,0.123589,0.258606,...,32,0.047383,0.347803,32,0.036936,0.360902,32,0.072983,0.394058,31


In [27]:
(new_para_new_para['new_para_cor_p_baseline']  - new_para_new_para['subset_new_para_cor_p_baseline']).describe().round(3)

count    954.000
mean       0.017
std        0.157
min       -0.610
25%       -0.049
50%        0.000
75%        0.085
max        0.655
dtype: float64

## age

In [28]:
new_para_new_para[(new_para_new_para['new_para_mv_age_p_baseline'] < 0.05) & (new_para_new_para['subset_new_para_mv_age_p_baseline'] >= 0.05)]

,var1,var2,new_para_cor_p_baseline,new_para_cor_r_baseline,new_para_cor_n,new_para_mv_age_p_baseline,new_para_mv_age_B_baseline,new_para_mv_age_n,new_para_mv_ageedu_p_baseline,new_para_mv_ageedu_B_baseline,...,subset_new_para_cor_n,subset_new_para_mv_age_p_baseline,subset_new_para_mv_age_B_baseline,subset_new_para_mv_age_n,subset_new_para_mv_ageedu_p_baseline,subset_new_para_mv_ageedu_B_baseline,subset_new_para_mv_ageedu_n,subset_new_para_mv_aeasr_p_baseline,subset_new_para_mv_aeasr_B_baseline,subset_new_para_mv_aeasr_n
32,GFAP,FRONTAL_gm,0.063296,-0.289129,42,0.038012,-0.331277,41,0.116266,-0.263333,...,37,0.053991,-0.320980,37,0.095089,-0.300080,37,0.051380,-0.370385,36
34,GFAP,Precun_gm,0.182021,-0.209959,42,0.042803,-0.328679,41,0.141319,-0.254570,...,37,0.078606,-0.296121,37,0.134712,-0.270903,37,0.115938,-0.304916,36
36,GFAP,RASref1_gm,0.026651,-0.341963,42,0.018361,-0.394031,41,0.065472,-0.326914,...,37,0.060628,-0.316971,37,0.111821,-0.301050,37,0.102489,-0.328714,36
360,Ab42,VOL_SupTemp_Rz,0.049328,-0.361998,30,0.035937,-0.383221,30,0.025722,-0.406561,...,26,0.062611,-0.370157,26,0.041274,-0.406264,26,0.007725,-0.578155,25
368,Ab42,VOL_SupraMarg_Rz,0.032356,-0.391590,30,0.035204,-0.385309,30,0.042557,-0.372632,...,26,0.062031,-0.370998,26,0.082247,-0.350274,26,0.010062,-0.621520,25
403,Ab42,VOL_LatTemp_Rz,0.054639,-0.354433,30,0.045267,-0.367921,30,0.018458,-0.438603,...,26,0.069618,-0.370553,26,0.041626,-0.417958,26,0.005060,-0.631218,25
544,GFAP,VOL_Pericalc_Rz,0.007185,0.440412,36,0.027934,0.373309,36,0.014405,0.409138,...,32,0.086157,0.312673,32,0.062553,0.351801,32,0.037099,0.421684,31
576,GFAP,VOL_InfParSupra_Lz,0.141524,-0.249936,36,0.038333,-0.411740,36,0.042965,-0.397833,...,32,0.073053,-0.369076,32,0.082608,-0.363739,32,0.057829,-0.423614,31
733,pTau217,VOL_Ventricles_Rz,0.127420,0.275186,32,0.049041,0.347275,32,0.054523,0.336066,...,29,0.090469,0.321748,29,0.091997,0.322891,29,0.160988,0.298221,28
834,pTau231,VOL_OrbitFront_Rz,0.058212,0.338343,32,0.043712,0.362945,32,0.033779,0.378601,...,28,0.066237,0.358530,28,0.059541,0.372190,28,0.157077,0.319658,27


In [29]:
new_para_new_para[(new_para_new_para['new_para_mv_age_p_baseline'] >= 0.05) & (new_para_new_para['subset_new_para_mv_age_p_baseline'] < 0.05)]

,var1,var2,new_para_cor_p_baseline,new_para_cor_r_baseline,new_para_cor_n,new_para_mv_age_p_baseline,new_para_mv_age_B_baseline,new_para_mv_age_n,new_para_mv_ageedu_p_baseline,new_para_mv_ageedu_B_baseline,...,subset_new_para_cor_n,subset_new_para_mv_age_p_baseline,subset_new_para_mv_age_B_baseline,subset_new_para_mv_age_n,subset_new_para_mv_ageedu_p_baseline,subset_new_para_mv_ageedu_B_baseline,subset_new_para_mv_ageedu_n,subset_new_para_mv_aeasr_p_baseline,subset_new_para_mv_aeasr_B_baseline,subset_new_para_mv_aeasr_n
1,Ab40,SensMot_gm,0.061871,0.297936,40,0.068632,0.299515,39,0.076043,0.295868,...,35,0.031817,0.370376,35,0.036041,0.371019,35,0.025934,0.413377,34
11,Ab42,SensMot_gm,0.057967,0.318939,36,0.099368,0.276843,35,0.088305,0.284968,...,31,0.044969,0.370545,31,0.049590,0.362814,31,0.082676,0.367633,30
35,GFAP,Par_gm,0.167936,-0.216766,42,0.073024,-0.302208,41,0.130108,-0.254096,...,37,0.039692,-0.358004,37,0.063931,-0.335419,37,0.071454,-0.338072,36
59,pTau181,PostCing_gm,0.419342,0.127958,42,0.052931,0.304133,41,0.004673,0.441786,...,37,0.034828,0.349197,37,0.003891,0.477323,37,0.002768,0.541936,36
61,pTau217,SensMot_gm,0.038304,-0.337401,38,0.053055,-0.316313,37,0.050653,-0.320993,...,34,0.008904,-0.455977,34,0.010656,-0.453336,34,0.024095,-0.448091,33
128,NFL,pTau231,0.176397,0.220985,39,0.074058,0.307459,36,0.083070,0.314042,...,34,0.030106,0.386122,32,0.034441,0.394967,32,0.069114,0.358994,31
150,pTau231,NFL,0.176397,0.220985,39,0.074058,0.304068,36,0.083070,0.289550,...,34,0.030106,0.393848,32,0.034441,0.379902,32,0.069114,0.365184,31
390,Ab42,VOL_Pericalc_Rz,0.149906,0.269440,30,0.075138,0.328842,30,0.073484,0.330155,...,26,0.047829,0.392090,26,0.055751,0.380619,26,0.037794,0.463328,25
400,Ab42,VOL_TotalGray_Lz,0.274176,-0.206252,30,0.066666,-0.370421,30,0.053449,-0.389353,...,26,0.032601,-0.463715,26,0.016717,-0.520422,26,0.003976,-0.669727,25
406,Ab42,VOL_Frontal_Lz,0.178986,-0.252086,30,0.101208,-0.327468,30,0.108633,-0.320693,...,26,0.026896,-0.506957,26,0.010528,-0.589653,26,0.003907,-0.712971,25


In [30]:
(new_para_new_para['new_para_mv_age_p_baseline']  - new_para_new_para['subset_new_para_mv_age_p_baseline']).describe().round(3)

count    954.000
mean       0.003
std        0.139
min       -0.665
25%       -0.052
50%        0.000
75%        0.049
max        0.677
dtype: float64

## age education

In [31]:
new_para_new_para[(new_para_new_para['new_para_mv_ageedu_p_baseline'] < 0.05) & (new_para_new_para['subset_new_para_mv_ageedu_p_baseline'] >= 0.05)]

,var1,var2,new_para_cor_p_baseline,new_para_cor_r_baseline,new_para_cor_n,new_para_mv_age_p_baseline,new_para_mv_age_B_baseline,new_para_mv_age_n,new_para_mv_ageedu_p_baseline,new_para_mv_ageedu_B_baseline,...,subset_new_para_cor_n,subset_new_para_mv_age_p_baseline,subset_new_para_mv_age_B_baseline,subset_new_para_mv_age_n,subset_new_para_mv_ageedu_p_baseline,subset_new_para_mv_ageedu_B_baseline,subset_new_para_mv_ageedu_n,subset_new_para_mv_aeasr_p_baseline,subset_new_para_mv_aeasr_B_baseline,subset_new_para_mv_aeasr_n
52,pTau181,FRONTAL_gm,0.172523,0.214506,42,0.199865,0.204362,41,0.031049,0.348622,...,37,0.297231,0.177405,37,0.074760,0.312923,37,0.094632,0.322854,36
286,Ab40,VOL_InfPar_Lz,0.073481,-0.310924,34,0.052674,-0.385510,34,0.038276,-0.422644,...,30,0.076169,-0.369130,30,0.057413,-0.411009,30,0.087391,-0.399509,29
337,Ab40,VOL_Inf_Mid_Fus_Temp_Lz,0.058722,-0.327426,34,0.056431,-0.347208,34,0.039202,-0.386115,...,30,0.068366,-0.348004,30,0.055825,-0.376350,30,0.083807,-0.380138,29
354,Ab42,VOL_Fusi_Rz,0.049328,-0.361998,30,0.054651,-0.354205,30,0.026585,-0.412502,...,26,0.091279,-0.348408,26,0.072035,-0.371034,26,0.036197,-0.471892,25
356,Ab42,VOL_InfTemp_Rz,0.028494,-0.400045,30,0.057331,-0.353523,30,0.021506,-0.435866,...,26,0.093950,-0.350923,26,0.055934,-0.403455,26,0.009428,-0.599165,25
368,Ab42,VOL_SupraMarg_Rz,0.032356,-0.391590,30,0.035204,-0.385309,30,0.042557,-0.372632,...,26,0.062031,-0.370998,26,0.082247,-0.350274,26,0.010062,-0.621520,25
401,Ab42,VOL_TotalGray_Rz,0.114915,-0.293915,30,0.074008,-0.339201,30,0.037851,-0.398449,...,26,0.088098,-0.343521,26,0.053734,-0.390836,26,0.012319,-0.557134,25
409,Ab42,VOL_InfMidTemp_Rz,0.075167,-0.329736,30,0.107173,-0.300601,30,0.045060,-0.383478,...,26,0.136629,-0.312159,26,0.084574,-0.365269,26,0.006003,-0.687193,25
415,Ab42,VOL_Inf_Mid_Fus_Temp_Rz,0.077701,-0.327066,30,0.070734,-0.335025,30,0.027336,-0.417957,...,26,0.096751,-0.346905,26,0.060516,-0.394387,26,0.007286,-0.638132,25
498,Ab42_40,VOL_LatOccLingCun_Rz,0.164409,0.260512,30,0.054669,0.350490,30,0.047735,0.372335,...,26,0.055432,0.384304,26,0.060979,0.384009,26,0.179817,0.287735,25


In [32]:
new_para_new_para[(new_para_new_para['new_para_mv_ageedu_p_baseline'] >= 0.05) & (new_para_new_para['subset_new_para_mv_ageedu_p_baseline'] < 0.05)]

,var1,var2,new_para_cor_p_baseline,new_para_cor_r_baseline,new_para_cor_n,new_para_mv_age_p_baseline,new_para_mv_age_B_baseline,new_para_mv_age_n,new_para_mv_ageedu_p_baseline,new_para_mv_ageedu_B_baseline,...,subset_new_para_cor_n,subset_new_para_mv_age_p_baseline,subset_new_para_mv_age_B_baseline,subset_new_para_mv_age_n,subset_new_para_mv_ageedu_p_baseline,subset_new_para_mv_ageedu_B_baseline,subset_new_para_mv_ageedu_n,subset_new_para_mv_aeasr_p_baseline,subset_new_para_mv_aeasr_B_baseline,subset_new_para_mv_aeasr_n
1,Ab40,SensMot_gm,0.061871,0.297936,40,0.068632,0.299515,39,0.076043,0.295868,...,35,0.031817,0.370376,35,0.036041,0.371019,35,0.025934,0.413377,34
11,Ab42,SensMot_gm,0.057967,0.318939,36,0.099368,0.276843,35,0.088305,0.284968,...,31,0.044969,0.370545,31,0.049590,0.362814,31,0.082676,0.367633,30
61,pTau217,SensMot_gm,0.038304,-0.337401,38,0.053055,-0.316313,37,0.050653,-0.320993,...,34,0.008904,-0.455977,34,0.010656,-0.453336,34,0.024095,-0.448091,33
128,NFL,pTau231,0.176397,0.220985,39,0.074058,0.307459,36,0.083070,0.314042,...,34,0.030106,0.386122,32,0.034441,0.394967,32,0.069114,0.358994,31
150,pTau231,NFL,0.176397,0.220985,39,0.074058,0.304068,36,0.083070,0.289550,...,34,0.030106,0.393848,32,0.034441,0.379902,32,0.069114,0.365184,31
375,Ab42,VOL_InfFront_Lz,0.190332,-0.245856,30,0.112209,-0.300371,30,0.069335,-0.346041,...,26,0.080306,-0.362181,26,0.017816,-0.514800,26,0.011333,-0.591016,25
400,Ab42,VOL_TotalGray_Lz,0.274176,-0.206252,30,0.066666,-0.370421,30,0.053449,-0.389353,...,26,0.032601,-0.463715,26,0.016717,-0.520422,26,0.003976,-0.669727,25
406,Ab42,VOL_Frontal_Lz,0.178986,-0.252086,30,0.101208,-0.327468,30,0.108633,-0.320693,...,26,0.026896,-0.506957,26,0.010528,-0.589653,26,0.003907,-0.712971,25
407,Ab42,VOL_Frontal_Rz,0.110940,-0.297030,30,0.062376,-0.352297,30,0.054998,-0.361839,...,26,0.051923,-0.398309,26,0.031668,-0.441119,26,0.006759,-0.613134,25
506,GFAP,VOL_ParaHip_Rz,0.191293,0.222909,36,0.148370,0.249701,36,0.059296,0.327759,...,32,0.022871,0.402493,32,0.016516,0.435375,32,0.021614,0.433801,31


In [33]:
(new_para_new_para['new_para_mv_ageedu_p_baseline']  - new_para_new_para['subset_new_para_mv_ageedu_p_baseline']).describe().round(3)

count    954.000
mean      -0.001
std        0.130
min       -0.611
25%       -0.050
50%        0.000
75%        0.037
max        0.670
dtype: float64

## age education apoe sex race

In [34]:
new_para_new_para[(new_para_new_para['new_para_mv_aeasr_p_baseline'] < 0.05) & (new_para_new_para['subset_new_para_mv_aeasr_p_baseline'] >= 0.05)]

,var1,var2,new_para_cor_p_baseline,new_para_cor_r_baseline,new_para_cor_n,new_para_mv_age_p_baseline,new_para_mv_age_B_baseline,new_para_mv_age_n,new_para_mv_ageedu_p_baseline,new_para_mv_ageedu_B_baseline,...,subset_new_para_cor_n,subset_new_para_mv_age_p_baseline,subset_new_para_mv_age_B_baseline,subset_new_para_mv_age_n,subset_new_para_mv_ageedu_p_baseline,subset_new_para_mv_ageedu_B_baseline,subset_new_para_mv_ageedu_n,subset_new_para_mv_aeasr_p_baseline,subset_new_para_mv_aeasr_B_baseline,subset_new_para_mv_aeasr_n


In [35]:
new_para_new_para[(new_para_new_para['new_para_mv_aeasr_p_baseline'] >= 0.05) & (new_para_new_para['subset_new_para_mv_aeasr_p_baseline'] < 0.05)]

,var1,var2,new_para_cor_p_baseline,new_para_cor_r_baseline,new_para_cor_n,new_para_mv_age_p_baseline,new_para_mv_age_B_baseline,new_para_mv_age_n,new_para_mv_ageedu_p_baseline,new_para_mv_ageedu_B_baseline,...,subset_new_para_cor_n,subset_new_para_mv_age_p_baseline,subset_new_para_mv_age_B_baseline,subset_new_para_mv_age_n,subset_new_para_mv_ageedu_p_baseline,subset_new_para_mv_ageedu_B_baseline,subset_new_para_mv_ageedu_n,subset_new_para_mv_aeasr_p_baseline,subset_new_para_mv_aeasr_B_baseline,subset_new_para_mv_aeasr_n


In [36]:
(new_para_new_para['new_para_mv_aeasr_p_baseline']  - new_para_new_para['subset_new_para_mv_aeasr_p_baseline']).describe().round(3)

count    954.0
mean      -0.0
std        0.0
min       -0.0
25%       -0.0
50%        0.0
75%        0.0
max        0.0
dtype: float64

# Ras vs new para

In [37]:
ras_new_para = pd.merge(results_ras, results_new_para, how = 'inner')
ras_new_para

,var1,var2,ras_cor_p_baseline,ras_cor_r_baseline,ras_cor_n,ras_mv_age_p_baseline,ras_mv_age_B_baseline,ras_mv_age_n,ras_mv_ageedu_p_baseline,ras_mv_ageedu_B_baseline,...,new_para_cor_n,new_para_mv_age_p_baseline,new_para_mv_age_B_baseline,new_para_mv_age_n,new_para_mv_ageedu_p_baseline,new_para_mv_ageedu_B_baseline,new_para_mv_ageedu_n,new_para_mv_aeasr_p_baseline,new_para_mv_aeasr_B_baseline,new_para_mv_aeasr_n
0,Ab40,MTL_gm,0.223522,-0.196811,40,0.489179,-0.118124,39,0.439274,-0.135441,...,40,0.888369,-0.024268,39,0.682126,-0.081689,39,0.998305,0.000562,34
1,Ab40,SensMot_gm,0.160693,0.226079,40,0.732204,0.060247,39,0.611434,0.095214,...,40,0.068632,0.299515,39,0.076043,0.295868,39,0.025934,0.441393,34
2,Ab40,FRONTAL_gm,0.983500,0.003377,40,0.915039,-0.018576,39,0.909731,-0.019968,...,40,0.480051,0.118418,39,0.568897,0.104043,39,0.529761,0.139394,34
3,Ab40,AC_gm,0.722342,-0.057974,40,0.466839,-0.123735,39,0.476536,-0.122456,...,40,0.768005,-0.051412,39,0.672515,-0.077033,39,0.941134,0.017332,34
4,Ab40,Precun_gm,0.683026,-0.066604,40,0.488883,-0.123230,39,0.470999,-0.130127,...,40,0.940730,0.012692,39,0.905003,-0.022502,39,0.899370,-0.027883,34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
958,pTau181_Ab42,VOL_Precun_InfPar_Supramarg_Rz,0.511106,0.124805,30,0.108750,0.311451,30,0.001254,0.563194,...,30,0.108750,0.311451,30,0.001254,0.563194,30,0.003669,0.630301,25
959,pTau181_Ab42,VOL_LatOccLingCun_Lz,0.203086,0.239155,30,0.071093,0.331401,30,0.026065,0.362568,...,30,0.071093,0.331401,30,0.026065,0.362568,30,0.007344,0.512186,25
960,pTau181_Ab42,VOL_LatOccLingCun_Rz,0.117861,0.291657,30,0.075020,0.327445,30,0.009236,0.423286,...,30,0.075020,0.327445,30,0.009236,0.423286,30,0.009140,0.503818,25
961,pTau181_Ab42,VOL_InfParSupra_Lz,0.556047,0.111902,30,0.144645,0.306248,30,0.036780,0.390568,...,30,0.144645,0.306248,30,0.036780,0.390568,30,0.037092,0.463150,25


## cor

In [38]:
ras_new_para[(ras_new_para['ras_cor_p_baseline'] < 0.05) & (ras_new_para['new_para_cor_p_baseline'] >= 0.05)]

,var1,var2,ras_cor_p_baseline,ras_cor_r_baseline,ras_cor_n,ras_mv_age_p_baseline,ras_mv_age_B_baseline,ras_mv_age_n,ras_mv_ageedu_p_baseline,ras_mv_ageedu_B_baseline,...,new_para_cor_n,new_para_mv_age_p_baseline,new_para_mv_age_B_baseline,new_para_mv_age_n,new_para_mv_ageedu_p_baseline,new_para_mv_ageedu_B_baseline,new_para_mv_ageedu_n,new_para_mv_aeasr_p_baseline,new_para_mv_aeasr_B_baseline,new_para_mv_aeasr_n
24,Ab42_40,Precun_gm,0.026167,0.370399,36,0.009156,0.464693,35,0.010441,0.465158,...,36,0.026605,0.376949,35,0.021403,0.425998,35,0.008954,0.517658,30


In [39]:
ras_new_para[(ras_new_para['ras_cor_p_baseline'] >= 0.05) & (ras_new_para['new_para_cor_p_baseline'] < 0.05)]

,var1,var2,ras_cor_p_baseline,ras_cor_r_baseline,ras_cor_n,ras_mv_age_p_baseline,ras_mv_age_B_baseline,ras_mv_age_n,ras_mv_ageedu_p_baseline,ras_mv_ageedu_B_baseline,...,new_para_cor_n,new_para_mv_age_p_baseline,new_para_mv_age_B_baseline,new_para_mv_age_n,new_para_mv_ageedu_p_baseline,new_para_mv_ageedu_B_baseline,new_para_mv_ageedu_n,new_para_mv_aeasr_p_baseline,new_para_mv_aeasr_B_baseline,new_para_mv_aeasr_n
25,Ab42_40,Par_gm,0.132430,0.255598,36,0.085827,0.312005,35,0.090301,0.312556,...,36,0.128642,0.262071,35,0.135750,0.264878,35,0.128537,0.284432,30
61,pTau217,SensMot_gm,0.151096,-0.237483,38,0.884184,-0.025813,37,0.704246,-0.070252,...,38,0.053055,-0.316313,37,0.050653,-0.320993,37,0.024095,-0.476497,33
64,pTau217,Precun_gm,0.142923,-0.242189,38,0.444568,-0.134744,37,0.498175,-0.120823,...,38,0.079768,-0.289403,37,0.120831,-0.274519,37,0.127457,-0.311953,33


In [40]:
(ras_new_para['ras_cor_p_baseline']  - ras_new_para['new_para_cor_p_baseline']).describe().round(3)

count    945.000
mean       0.004
std        0.082
min       -0.905
25%        0.000
50%        0.000
75%        0.000
max        0.758
dtype: float64

## age

In [41]:
ras_new_para.loc[(ras_new_para['ras_mv_age_p_baseline'] < 0.05) & (ras_new_para['new_para_mv_age_p_baseline'] >= 0.05), ['var1', 'var2',
'ras_mv_age_p_baseline', 'ras_mv_age_B_baseline', 'ras_mv_age_n',  'new_para_mv_age_p_baseline', 'new_para_mv_age_B_baseline',
       'new_para_mv_age_n']]

,var1,var2,ras_mv_age_p_baseline,ras_mv_age_B_baseline,ras_mv_age_n,new_para_mv_age_p_baseline,new_para_mv_age_B_baseline,new_para_mv_age_n
52,pTau181,FRONTAL_gm,0.040693,0.332369,41,0.199865,0.204362,41
59,pTau181,PostCing_gm,0.015023,0.389445,41,0.052931,0.304133,41
80,pTau181_Ab42,MTL_gm,0.032730,0.355620,35,0.199829,0.222713,35


In [42]:
ras_new_para.loc[(ras_new_para['ras_mv_age_p_baseline'] >= 0.05) & (ras_new_para['new_para_mv_age_p_baseline'] < 0.05), ['var1', 'var2',
'ras_mv_age_p_baseline', 'ras_mv_age_B_baseline', 'ras_mv_age_n',  'new_para_mv_age_p_baseline', 'new_para_mv_age_B_baseline',
       'new_para_mv_age_n']]

,var1,var2,ras_mv_age_p_baseline,ras_mv_age_B_baseline,ras_mv_age_n,new_para_mv_age_p_baseline,new_para_mv_age_B_baseline,new_para_mv_age_n
32,GFAP,FRONTAL_gm,0.941449,-0.01250,41,0.038012,-0.331277,41
34,GFAP,Precun_gm,0.840350,-0.03517,41,0.042803,-0.328679,41
67,pTau217,Temp_gm,0.059497,-0.31941,37,0.002100,-0.479222,37


In [43]:
(ras_new_para['ras_mv_age_p_baseline']  - ras_new_para['new_para_mv_age_p_baseline']).describe().round(3)

count    945.000
mean       0.001
std        0.111
min       -0.771
25%        0.000
50%        0.000
75%        0.000
max        0.903
dtype: float64

## age education

In [44]:
ras_new_para[(ras_new_para['ras_mv_ageedu_p_baseline'] < 0.05) & (ras_new_para['new_para_mv_ageedu_p_baseline'] >= 0.05)]

,var1,var2,ras_cor_p_baseline,ras_cor_r_baseline,ras_cor_n,ras_mv_age_p_baseline,ras_mv_age_B_baseline,ras_mv_age_n,ras_mv_ageedu_p_baseline,ras_mv_ageedu_B_baseline,...,new_para_cor_n,new_para_mv_age_p_baseline,new_para_mv_age_B_baseline,new_para_mv_age_n,new_para_mv_ageedu_p_baseline,new_para_mv_ageedu_B_baseline,new_para_mv_ageedu_n,new_para_mv_aeasr_p_baseline,new_para_mv_aeasr_B_baseline,new_para_mv_aeasr_n
69,pTau217,PostCing_gm,0.851507,-0.031409,38,0.051007,0.330735,37,0.037717,0.355517,...,38,0.164914,0.228607,37,0.072173,0.314456,37,0.197719,0.267418,33


In [45]:
ras_new_para[(ras_new_para['ras_mv_ageedu_p_baseline'] >= 0.05) & (ras_new_para['new_para_mv_ageedu_p_baseline'] < 0.05)]

,var1,var2,ras_cor_p_baseline,ras_cor_r_baseline,ras_cor_n,ras_mv_age_p_baseline,ras_mv_age_B_baseline,ras_mv_age_n,ras_mv_ageedu_p_baseline,ras_mv_ageedu_B_baseline,...,new_para_cor_n,new_para_mv_age_p_baseline,new_para_mv_age_B_baseline,new_para_mv_age_n,new_para_mv_ageedu_p_baseline,new_para_mv_ageedu_B_baseline,new_para_mv_ageedu_n,new_para_mv_aeasr_p_baseline,new_para_mv_aeasr_B_baseline,new_para_mv_aeasr_n
67,pTau217,Temp_gm,0.030900,-0.350643,38,0.059497,-0.319410,37,0.061400,-0.318629,...,38,0.00210,-0.479222,37,0.003303,-0.470622,37,0.001588,-0.621721,33
79,pTau231,PostCing_gm,0.245891,0.192910,38,0.066422,0.322885,37,0.061674,0.331900,...,38,0.06636,0.311201,37,0.031009,0.385201,37,0.011149,0.509237,32


In [46]:
(ras_new_para['ras_mv_ageedu_p_baseline']  - ras_new_para['new_para_mv_ageedu_p_baseline']).describe().round(3)

count    945.000
mean       0.002
std        0.090
min       -0.680
25%        0.000
50%        0.000
75%        0.000
max        0.857
dtype: float64

## age education apoe sex race

In [47]:
ras_new_para[(ras_new_para['ras_mv_aeasr_p_baseline'] < 0.05) & (ras_new_para['new_para_mv_aeasr_p_baseline'] >= 0.05)]

,var1,var2,ras_cor_p_baseline,ras_cor_r_baseline,ras_cor_n,ras_mv_age_p_baseline,ras_mv_age_B_baseline,ras_mv_age_n,ras_mv_ageedu_p_baseline,ras_mv_ageedu_B_baseline,...,new_para_cor_n,new_para_mv_age_p_baseline,new_para_mv_age_B_baseline,new_para_mv_age_n,new_para_mv_ageedu_p_baseline,new_para_mv_ageedu_B_baseline,new_para_mv_ageedu_n,new_para_mv_aeasr_p_baseline,new_para_mv_aeasr_B_baseline,new_para_mv_aeasr_n


In [48]:
ras_new_para[(ras_new_para['ras_mv_aeasr_p_baseline'] >= 0.05) & (ras_new_para['new_para_mv_aeasr_p_baseline'] < 0.05)]

,var1,var2,ras_cor_p_baseline,ras_cor_r_baseline,ras_cor_n,ras_mv_age_p_baseline,ras_mv_age_B_baseline,ras_mv_age_n,ras_mv_ageedu_p_baseline,ras_mv_ageedu_B_baseline,...,new_para_cor_n,new_para_mv_age_p_baseline,new_para_mv_age_B_baseline,new_para_mv_age_n,new_para_mv_ageedu_p_baseline,new_para_mv_ageedu_B_baseline,new_para_mv_ageedu_n,new_para_mv_aeasr_p_baseline,new_para_mv_aeasr_B_baseline,new_para_mv_aeasr_n
1,Ab40,SensMot_gm,0.160693,0.226079,40,0.732204,0.060247,39,0.611434,0.095214,...,40,0.068632,0.299515,39,0.076043,0.295868,39,0.025934,0.441393,34
61,pTau217,SensMot_gm,0.151096,-0.237483,38,0.884184,-0.025813,37,0.704246,-0.070252,...,38,0.053055,-0.316313,37,0.050653,-0.320993,37,0.024095,-0.476497,33


In [49]:
(ras_new_para['ras_mv_aeasr_p_baseline']  - ras_new_para['new_para_mv_aeasr_p_baseline']).describe().round(3)

count    945.000
mean       0.004
std        0.064
min       -0.554
25%        0.000
50%        0.000
75%        0.000
max        0.874
dtype: float64

# Composite vs Old Para

In [50]:
comp_para = pd.merge(results_composite, results_para, how = 'inner')
comp_para

,var1,var2,composite_cor_p_baseline,composite_cor_r_baseline,composite_cor_n,composite_mv_age_p_baseline,composite_mv_age_B_baseline,composite_mv_age_n,composite_mv_ageedu_p_baseline,composite_mv_ageedu_B_baseline,...,para_cor_n,para_mv_age_p_baseline,para_mv_age_B_baseline,para_mv_age_n,para_mv_ageedu_p_baseline,para_mv_ageedu_B_baseline,para_mv_ageedu_n,para_mv_aeasr_p_baseline,para_mv_aeasr_B_baseline,para_mv_aeasr_n
0,Ab40,Avg_PCC,0.894536,0.023249,35,0.675199,0.074996,35,0.737551,0.066836,...,35,0.952775,-0.010800,35,0.919046,-0.019063,35,0.830950,-0.042220,34
1,Ab40,MTL_gm,0.248673,-0.200280,35,0.878973,-0.029233,35,0.750044,-0.071179,...,35,0.548743,-0.108591,35,0.525368,-0.117922,35,0.611618,-0.103449,34
2,Ab40,AC_gm,0.725668,-0.061489,35,0.991560,0.001962,35,0.940486,-0.014797,...,35,0.657984,-0.079269,35,0.657049,-0.080733,35,0.683330,-0.079590,34
3,Ab40,Precun_gm,0.914754,-0.018774,35,0.910323,-0.020120,35,0.824361,-0.043250,...,35,0.523210,-0.115307,35,0.512521,-0.120524,35,0.544580,-0.117307,34
4,Ab40,Par_gm,0.957331,0.009384,35,0.793883,0.048686,35,0.846295,0.038114,...,35,0.956933,-0.010622,35,0.936615,-0.015957,35,0.961421,0.010141,34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
931,pTau181_Ab42,VOL_Precun_InfPar_Supramarg_Rz,0.511106,0.124805,30,0.108750,0.311451,30,0.001254,0.563194,...,30,0.108750,0.311451,30,0.001254,0.563194,30,0.003669,0.630301,25
932,pTau181_Ab42,VOL_LatOccLingCun_Lz,0.203086,0.239155,30,0.071093,0.331401,30,0.026065,0.362568,...,30,0.071093,0.331401,30,0.026065,0.362568,30,0.007344,0.512186,25
933,pTau181_Ab42,VOL_LatOccLingCun_Rz,0.117861,0.291657,30,0.075020,0.327445,30,0.009236,0.423286,...,30,0.075020,0.327445,30,0.009236,0.423286,30,0.009140,0.503818,25
934,pTau181_Ab42,VOL_InfParSupra_Lz,0.556047,0.111902,30,0.144645,0.306248,30,0.036780,0.390568,...,30,0.144645,0.306248,30,0.036780,0.390568,30,0.037092,0.463150,25


## cor

In [51]:
comp_para[(comp_para['composite_cor_p_baseline'] < 0.05) & (comp_para['para_cor_p_baseline'] >= 0.05)]

,var1,var2,composite_cor_p_baseline,composite_cor_r_baseline,composite_cor_n,composite_mv_age_p_baseline,composite_mv_age_B_baseline,composite_mv_age_n,composite_mv_ageedu_p_baseline,composite_mv_ageedu_B_baseline,...,para_cor_n,para_mv_age_p_baseline,para_mv_age_B_baseline,para_mv_age_n,para_mv_ageedu_p_baseline,para_mv_ageedu_B_baseline,para_mv_ageedu_n,para_mv_aeasr_p_baseline,para_mv_aeasr_B_baseline,para_mv_aeasr_n
18,Ab42_40,Par_gm,0.042040,0.367376,31,0.160287,0.264757,31,0.177073,0.265001,...,31,0.033869,0.411900,31,0.038248,0.409989,31,0.034491,0.402393,30
45,pTau217,Precun_gm,0.019056,-0.400076,34,0.087733,-0.302345,34,0.099155,-0.313984,...,34,0.363550,-0.164719,34,0.386465,-0.159947,34,0.520300,-0.126176,33
47,pTau217,Temp_gm,0.003879,-0.482192,34,0.000815,-0.547443,34,0.000943,-0.566491,...,34,0.026528,-0.382766,34,0.029604,-0.381035,34,0.065297,-0.352158,33
50,pTau231,MTL_gm,0.022761,0.395384,33,0.028650,0.422525,33,0.018043,0.492179,...,33,0.146443,0.270415,33,0.149712,0.273452,33,0.326453,0.212553,32


In [52]:
comp_para[(comp_para['composite_cor_p_baseline'] >= 0.05) & (comp_para['para_cor_p_baseline'] < 0.05)]

,var1,var2,composite_cor_p_baseline,composite_cor_r_baseline,composite_cor_n,composite_mv_age_p_baseline,composite_mv_age_B_baseline,composite_mv_age_n,composite_mv_ageedu_p_baseline,composite_mv_ageedu_B_baseline,...,para_cor_n,para_mv_age_p_baseline,para_mv_age_B_baseline,para_mv_age_n,para_mv_ageedu_p_baseline,para_mv_ageedu_B_baseline,para_mv_ageedu_n,para_mv_aeasr_p_baseline,para_mv_aeasr_B_baseline,para_mv_aeasr_n
17,Ab42_40,Precun_gm,0.057075,0.345335,31,0.021037,0.419073,31,0.020631,0.454900,...,31,0.002686,0.551934,31,0.003350,0.552870,31,0.003939,0.521551,30
41,pTau181,PostCing_gm,0.160375,0.235608,37,0.034887,0.349089,37,0.003914,0.477008,...,37,0.026778,0.366908,37,0.015564,0.390169,37,0.016755,0.409190,36


In [53]:
(comp_para['composite_cor_p_baseline']  - comp_para['para_cor_p_baseline']).describe().round(3)

count    936.000
mean      -0.002
std        0.054
min       -0.503
25%        0.000
50%        0.000
75%        0.000
max        0.493
dtype: float64

## age

In [54]:
comp_para.loc[(comp_para['composite_mv_age_p_baseline'] < 0.05) & (comp_para['para_mv_age_p_baseline'] >= 0.05), ['var1', 'var2', 'composite_mv_age_p_baseline',
       'composite_mv_age_B_baseline', 'composite_mv_age_n', 'para_mv_age_p_baseline', 'para_mv_age_B_baseline',
       'para_mv_age_n']]

,var1,var2,composite_mv_age_p_baseline,composite_mv_age_B_baseline,composite_mv_age_n,para_mv_age_p_baseline,para_mv_age_B_baseline,para_mv_age_n
25,GFAP,Par_gm,0.04006,-0.357448,37,0.068735,-0.335265,37
50,pTau231,MTL_gm,0.02865,0.422525,33,0.146443,0.270415,33


In [55]:
comp_para.columns

Index(['var1', 'var2', 'composite_cor_p_baseline', 'composite_cor_r_baseline',
       'composite_cor_n', 'composite_mv_age_p_baseline',
       'composite_mv_age_B_baseline', 'composite_mv_age_n',
       'composite_mv_ageedu_p_baseline', 'composite_mv_ageedu_B_baseline',
       'composite_mv_ageedu_n', 'composite_mv_aeasr_p_baseline',
       'composite_mv_aeasr_B_baseline', 'composite_mv_aeasr_n', 'type',
       'para_cor_p_baseline', 'para_cor_r_baseline', 'para_cor_n',
       'para_mv_age_p_baseline', 'para_mv_age_B_baseline', 'para_mv_age_n',
       'para_mv_ageedu_p_baseline', 'para_mv_ageedu_B_baseline',
       'para_mv_ageedu_n', 'para_mv_aeasr_p_baseline',
       'para_mv_aeasr_B_baseline', 'para_mv_aeasr_n'],
      dtype='object')

In [56]:
comp_para.loc[(comp_para['composite_mv_age_p_baseline'] >= 0.05) & (comp_para['para_mv_age_p_baseline'] < 0.05) ,['var1', 'var2','type', 'composite_mv_age_p_baseline', 'para_mv_age_p_baseline']]

,var1,var2,type,composite_mv_age_p_baseline,para_mv_age_p_baseline
14,Ab42_40,Avg_PCC,FDG PET,0.078268,0.015326
18,Ab42_40,Par_gm,FDG PET,0.160287,0.033869
19,Ab42_40,Temp_gm,FDG PET,0.151228,0.047517


In [57]:
(comp_para['composite_mv_age_p_baseline']  - comp_para['para_mv_age_p_baseline'])[(comp_para['composite_mv_age_p_baseline']  - comp_para['para_mv_age_p_baseline']) != 0].describe().round(3).round(2)

count    63.00
mean     -0.05
std       0.24
min      -0.53
25%      -0.25
50%      -0.04
75%       0.12
max       0.54
dtype: float64

In [58]:
(comp_para['composite_mv_age_p_baseline']  - comp_para['para_mv_age_p_baseline']).describe().round(3)

count    936.000
mean      -0.004
std        0.065
min       -0.529
25%        0.000
50%        0.000
75%        0.000
max        0.539
dtype: float64

In [59]:
(comp_para['composite_mv_age_n']  - comp_para['para_mv_age_n']).value_counts()

0    936
Name: count, dtype: int64

## age education

In [60]:
comp_para[(comp_para['composite_mv_ageedu_p_baseline'] < 0.05) & (comp_para['para_mv_ageedu_p_baseline'] >= 0.05)]

,var1,var2,composite_cor_p_baseline,composite_cor_r_baseline,composite_cor_n,composite_mv_age_p_baseline,composite_mv_age_B_baseline,composite_mv_age_n,composite_mv_ageedu_p_baseline,composite_mv_ageedu_B_baseline,...,para_cor_n,para_mv_age_p_baseline,para_mv_age_B_baseline,para_mv_age_n,para_mv_ageedu_p_baseline,para_mv_ageedu_B_baseline,para_mv_ageedu_n,para_mv_aeasr_p_baseline,para_mv_aeasr_B_baseline,para_mv_aeasr_n
50,pTau231,MTL_gm,0.022761,0.395384,33,0.028650,0.422525,33,0.018043,0.492179,...,33,0.146443,0.270415,33,0.149712,0.273452,33,0.326453,0.212553,32
55,pTau231,PostCing_gm,0.237497,0.211450,33,0.053393,0.345278,33,0.040653,0.389363,...,33,0.150270,0.261723,33,0.151998,0.266124,33,0.103545,0.313971,32
57,pTau181_Ab42,MTL_gm,0.159489,0.258975,31,0.274297,0.229314,31,0.035355,0.457403,...,31,0.246162,0.218392,31,0.171879,0.248776,31,0.293636,0.223343,30


In [61]:
comp_para[(comp_para['composite_mv_ageedu_p_baseline'] >= 0.05) & (comp_para['para_mv_ageedu_p_baseline'] < 0.05)]

,var1,var2,composite_cor_p_baseline,composite_cor_r_baseline,composite_cor_n,composite_mv_age_p_baseline,composite_mv_age_B_baseline,composite_mv_age_n,composite_mv_ageedu_p_baseline,composite_mv_ageedu_B_baseline,...,para_cor_n,para_mv_age_p_baseline,para_mv_age_B_baseline,para_mv_age_n,para_mv_ageedu_p_baseline,para_mv_ageedu_B_baseline,para_mv_ageedu_n,para_mv_aeasr_p_baseline,para_mv_aeasr_B_baseline,para_mv_aeasr_n
14,Ab42_40,Avg_PCC,0.083176,0.316129,31,0.078268,0.328191,31,0.078289,0.362323,...,31,0.015326,0.462449,31,0.017924,0.466623,31,0.009894,0.484942,30
18,Ab42_40,Par_gm,0.042040,0.367376,31,0.160287,0.264757,31,0.177073,0.265001,...,31,0.033869,0.411900,31,0.038248,0.409989,31,0.034491,0.402393,30


In [62]:
(comp_para['composite_mv_ageedu_p_baseline']  - comp_para['para_mv_ageedu_p_baseline']).describe().round(3)

count    936.000
mean      -0.003
std        0.050
min       -0.518
25%        0.000
50%        0.000
75%        0.000
max        0.473
dtype: float64

## age education apoe sex race

In [63]:
comp_para[(comp_para['composite_mv_aeasr_p_baseline'] < 0.05) & (comp_para['para_mv_aeasr_p_baseline'] >= 0.05)]

,var1,var2,composite_cor_p_baseline,composite_cor_r_baseline,composite_cor_n,composite_mv_age_p_baseline,composite_mv_age_B_baseline,composite_mv_age_n,composite_mv_ageedu_p_baseline,composite_mv_ageedu_B_baseline,...,para_cor_n,para_mv_age_p_baseline,para_mv_age_B_baseline,para_mv_age_n,para_mv_ageedu_p_baseline,para_mv_ageedu_B_baseline,para_mv_ageedu_n,para_mv_aeasr_p_baseline,para_mv_aeasr_B_baseline,para_mv_aeasr_n
47,pTau217,Temp_gm,0.003879,-0.482192,34,0.000815,-0.547443,34,0.000943,-0.566491,...,34,0.026528,-0.382766,34,0.029604,-0.381035,34,0.065297,-0.352158,33
49,pTau231,Avg_PCC,0.745054,0.058824,33,0.112249,0.304039,33,0.076309,0.375256,...,33,0.193251,0.252876,33,0.182737,0.268612,33,0.172006,0.285938,32
55,pTau231,PostCing_gm,0.237497,0.211450,33,0.053393,0.345278,33,0.040653,0.389363,...,33,0.150270,0.261723,33,0.151998,0.266124,33,0.103545,0.313971,32


In [64]:
comp_para[(comp_para['composite_mv_aeasr_p_baseline'] >= 0.05) & (comp_para['para_mv_aeasr_p_baseline'] < 0.05)]

,var1,var2,composite_cor_p_baseline,composite_cor_r_baseline,composite_cor_n,composite_mv_age_p_baseline,composite_mv_age_B_baseline,composite_mv_age_n,composite_mv_ageedu_p_baseline,composite_mv_ageedu_B_baseline,...,para_cor_n,para_mv_age_p_baseline,para_mv_age_B_baseline,para_mv_age_n,para_mv_ageedu_p_baseline,para_mv_ageedu_B_baseline,para_mv_ageedu_n,para_mv_aeasr_p_baseline,para_mv_aeasr_B_baseline,para_mv_aeasr_n
18,Ab42_40,Par_gm,0.04204,0.367376,31,0.160287,0.264757,31,0.177073,0.265001,...,31,0.033869,0.4119,31,0.038248,0.409989,31,0.034491,0.402393,30


In [65]:
(comp_para['composite_mv_aeasr_p_baseline']  - comp_para['para_mv_aeasr_p_baseline']).describe().round(3)

count    936.000
mean      -0.003
std        0.055
min       -0.484
25%        0.000
50%        0.000
75%        0.000
max        0.445
dtype: float64

# Within Composite covariate effects

## no covariates vs age

In [66]:
results_composite.columns

Index(['var1', 'var2', 'composite_cor_p_baseline', 'composite_cor_r_baseline',
       'composite_cor_n', 'composite_mv_age_p_baseline',
       'composite_mv_age_B_baseline', 'composite_mv_age_n',
       'composite_mv_ageedu_p_baseline', 'composite_mv_ageedu_B_baseline',
       'composite_mv_ageedu_n', 'composite_mv_aeasr_p_baseline',
       'composite_mv_aeasr_B_baseline', 'composite_mv_aeasr_n', 'type'],
      dtype='object')

In [67]:
# significance lost by adding covariates
results_composite.loc[(results_composite['composite_cor_p_baseline']<0.05) & 
(results_composite['composite_mv_age_p_baseline'] >= 0.05),
['var1', 'var2', 'composite_cor_p_baseline', 'composite_cor_r_baseline', 'composite_cor_n', 'composite_mv_age_p_baseline', 
'composite_mv_age_B_baseline', 'composite_mv_age_n', 'type']]

,var1,var2,composite_cor_p_baseline,composite_cor_r_baseline,composite_cor_n,composite_mv_age_p_baseline,composite_mv_age_B_baseline,composite_mv_age_n,type
35,Ab42_40,Par_gm,0.042040,0.367376,31,0.160287,0.264757,31,FDG PET
86,pTau217,Precun_gm,0.019056,-0.400076,34,0.087733,-0.302345,34,FDG PET
5,Ab40,pTau181,0.046590,-0.308833,42,0.141322,-0.249397,39,Plasma
13,Ab42,NFL,0.022278,0.369870,38,0.068675,0.310093,35,Plasma
16,Ab42,pTau231,0.047037,-0.333226,36,0.099934,-0.289168,33,Plasma
37,NFL,Ab42,0.022278,0.369870,38,0.068675,0.321993,35,Plasma
42,NFL,pTau217,0.023581,0.361901,39,0.295995,0.195039,37,Plasma
45,pTau181,Ab40,0.046590,-0.308833,42,0.141322,-0.237027,39,Plasma
58,pTau217,NFL,0.023581,0.361901,39,0.295995,0.164434,37,Plasma
61,pTau217,pTau231,0.012080,0.413953,36,0.157231,0.236624,34,Plasma


In [68]:
# significance gained by adding covariates
results_composite.loc[(results_composite['composite_cor_p_baseline']>=0.05) & 
(results_composite['composite_mv_age_p_baseline'] < 0.05),
['var1', 'var2', 'composite_cor_p_baseline', 'composite_cor_r_baseline', 'composite_cor_n', 'composite_mv_age_p_baseline', 
'composite_mv_age_B_baseline', 'composite_mv_age_n', 'type']]

,var1,var2,composite_cor_p_baseline,composite_cor_r_baseline,composite_cor_n,composite_mv_age_p_baseline,composite_mv_age_B_baseline,composite_mv_age_n,type
31,Ab42_40,L_Hip,0.133143,0.275806,31,0.044119,0.381478,31,FDG PET
34,Ab42_40,Precun_gm,0.057075,0.345335,31,0.021037,0.419073,31,FDG PET
48,GFAP,Par_gm,0.145801,-0.243865,37,0.040060,-0.357448,37,FDG PET
77,pTau181,PostCing_gm,0.160375,0.235608,37,0.034887,0.349089,37,FDG PET
23,Ab42_40,pTau181,0.164136,0.230344,38,0.004839,0.469415,35,Plasma
25,Ab42_40,pTau231,0.093330,0.283912,36,0.001587,0.538329,33,Plasma
26,Ab42_40,pTau181_Ab42,0.132487,0.248495,38,0.011804,0.428299,35,Plasma
41,NFL,pTau181,0.105834,0.247119,44,0.036299,0.330747,41,Plasma
47,pTau181,Ab42_40,0.164136,0.230344,38,0.004839,0.474380,35,Plasma
49,pTau181,NFL,0.105834,0.247119,44,0.036299,0.333445,41,Plasma


In [69]:
# difference in p-values
(results_composite['composite_cor_p_baseline'] - results_composite['composite_mv_age_p_baseline'] ).describe().round(3)

count    990.000
mean       0.022
std        0.209
min       -0.742
25%       -0.085
50%        0.007
75%        0.137
max        0.702
dtype: float64

## age vs age and education

In [70]:
results_composite.columns

Index(['var1', 'var2', 'composite_cor_p_baseline', 'composite_cor_r_baseline',
       'composite_cor_n', 'composite_mv_age_p_baseline',
       'composite_mv_age_B_baseline', 'composite_mv_age_n',
       'composite_mv_ageedu_p_baseline', 'composite_mv_ageedu_B_baseline',
       'composite_mv_ageedu_n', 'composite_mv_aeasr_p_baseline',
       'composite_mv_aeasr_B_baseline', 'composite_mv_aeasr_n', 'type'],
      dtype='object')

In [71]:
# significance lost by adding covariates
results_composite.loc[(results_composite['composite_mv_age_p_baseline']<0.05) & 
(results_composite['composite_mv_ageedu_p_baseline'] >= 0.05),
['var1', 'var2', 'composite_mv_age_p_baseline',  'composite_mv_age_B_baseline', 'composite_mv_age_n',
       'composite_mv_ageedu_p_baseline', 'composite_mv_ageedu_B_baseline',  'composite_mv_ageedu_n', 'type']]

,var1,var2,composite_mv_age_p_baseline,composite_mv_age_B_baseline,composite_mv_age_n,composite_mv_ageedu_p_baseline,composite_mv_ageedu_B_baseline,composite_mv_ageedu_n,type
48,GFAP,Par_gm,0.040060,-0.357448,37,0.064486,-0.334816,37,FDG PET
64,pTau181,tma,0.041674,0.378808,34,0.056081,0.345504,34,Cognitive
282,GFAP,VOL_Hip_Rz,0.046973,0.339147,36,0.053214,0.326464,36,Volumetric
463,pTau217,VOL_Ventricles_Rz,0.049041,0.347275,32,0.054523,0.336066,32,Volumetric
519,pTau217,VOL_Parietal_Lz,0.045699,-0.412356,32,0.066935,-0.379981,32,Volumetric


In [72]:
# significance gained by adding covariates
results_composite.loc[(results_composite['composite_mv_age_p_baseline']>=0.05) & 
(results_composite['composite_mv_ageedu_p_baseline'] < 0.05),
['var1', 'var2', 'composite_mv_ageedu_B_baseline',  'composite_mv_ageedu_n', 'type']]

,var1,var2,composite_mv_ageedu_B_baseline,composite_mv_ageedu_n,type
76,pTau181,R_Hip,0.472835,37,FDG PET
93,pTau231,Avg_Hip,0.413200,33,FDG PET
103,pTau231,PostCing_gm,0.389363,33,FDG PET
107,pTau181_Ab42,MTL_gm,0.457403,31,FDG PET
53,NFL,tmb,0.477913,19,Cognitive
...,...,...,...,...,...
688,pTau181_Ab42,VOL_Precun_InfPar_Supramarg_Rz,0.563194,30,Volumetric
689,pTau181_Ab42,VOL_LatOccLingCun_Lz,0.362568,30,Volumetric
690,pTau181_Ab42,VOL_LatOccLingCun_Rz,0.423286,30,Volumetric
691,pTau181_Ab42,VOL_InfParSupra_Lz,0.390568,30,Volumetric


In [73]:
# difference in p-values
(results_composite['composite_mv_age_p_baseline'] - results_composite['composite_mv_ageedu_p_baseline'] ).describe().round(3)

count    990.000
mean       0.020
std        0.126
min       -0.473
25%       -0.034
50%        0.006
75%        0.067
max        0.588
dtype: float64

## age and education vs all 5 (age, education, apoe, sex, race)

In [74]:
# significance lost by adding covariates
results_composite.loc[(results_composite['composite_mv_ageedu_p_baseline']<0.05) & 
(results_composite['composite_mv_aeasr_p_baseline'] >= 0.05),
['var1', 'var2', 'composite_mv_ageedu_p_baseline', 'composite_mv_ageedu_B_baseline',
       'composite_mv_ageedu_n', 'composite_mv_aeasr_p_baseline',
       'composite_mv_aeasr_B_baseline', 'composite_mv_aeasr_n', 'type']]

,var1,var2,composite_mv_ageedu_p_baseline,composite_mv_ageedu_B_baseline,composite_mv_ageedu_n,composite_mv_aeasr_p_baseline,composite_mv_aeasr_B_baseline,composite_mv_aeasr_n,type
29,Ab42_40,MTL_gm,0.015707,0.542670,31,0.095465,0.422335,30,FDG PET
31,Ab42_40,L_Hip,0.047296,0.400613,31,0.184307,0.287280,30,FDG PET
93,pTau231,Avg_Hip,0.045287,0.413200,33,0.118723,0.372945,32,FDG PET
94,pTau231,MTL_gm,0.018043,0.492179,33,0.071437,0.456563,32,FDG PET
107,pTau181_Ab42,MTL_gm,0.035355,0.457403,31,0.061005,0.501451,30,FDG PET
33,GFAP,pTau217,0.026726,0.408415,37,0.115845,0.317162,33,Plasma
57,pTau217,GFAP,0.026726,0.343122,37,0.115845,0.291276,33,Plasma
53,NFL,tmb,0.048453,0.477913,19,0.117391,0.453023,19,Cognitive
10,Ab40,VOL_MidTemp_Lz,0.033639,-0.419898,34,0.074042,-0.452477,29,Volumetric
16,Ab40,VOL_InfPar_Lz,0.038276,-0.422644,34,0.087391,-0.399884,29,Volumetric


In [75]:
# significance gained by adding covariates
results_composite.loc[(results_composite['composite_mv_ageedu_p_baseline']>=0.05) & 
(results_composite['composite_mv_aeasr_p_baseline'] < 0.05),
['var1', 'var2', 'composite_mv_ageedu_p_baseline', 'composite_mv_ageedu_B_baseline',
       'composite_mv_ageedu_n', 'composite_mv_aeasr_p_baseline',
       'composite_mv_aeasr_B_baseline', 'composite_mv_aeasr_n', 'type']]

,var1,var2,composite_mv_ageedu_p_baseline,composite_mv_ageedu_B_baseline,composite_mv_ageedu_n,composite_mv_aeasr_p_baseline,composite_mv_aeasr_B_baseline,composite_mv_aeasr_n,type
27,Ab42_40,Avg_PCC,0.078289,0.362323,31,0.017530,0.491478,30,FDG PET
92,pTau231,Avg_PCC,0.076309,0.375256,33,0.048050,0.451187,32,FDG PET
78,pTau217,cowattotal,0.108647,-0.296203,34,0.035282,-0.430309,33,Cognitive
85,Ab42,VOL_InfTemp_Lz,0.079282,-0.332517,30,0.023488,-0.529657,25,Volumetric
88,Ab42,VOL_MidTemp_Rz,0.161545,-0.269894,30,0.015840,-0.749644,25,Volumetric
94,Ab42,VOL_InfPar_Rz,0.071140,-0.389029,30,0.008457,-0.776933,25,Volumetric
105,Ab42,VOL_InfFront_Lz,0.069335,-0.346041,30,0.011333,-0.581402,25,Volumetric
107,Ab42,VOL_MidFront_Lz,0.270771,-0.240442,30,0.033886,-0.582940,25,Volumetric
110,Ab42,VOL_SupFront_Rz,0.251826,-0.227472,30,0.047335,-0.520237,25,Volumetric
120,Ab42,VOL_Pericalc_Rz,0.073484,0.330155,30,0.037794,0.469488,25,Volumetric


In [76]:
# difference in p-values
(results_composite['composite_mv_ageedu_p_baseline'] - results_composite['composite_mv_aeasr_p_baseline'] ).describe().round(3)

count    990.000
mean      -0.004
std        0.157
min       -0.621
25%       -0.082
50%       -0.001
75%        0.064
max        0.685
dtype: float64

# Within New Para covariate effects

## no covariates vs age

In [77]:
# significance lost by adding covariates
results_new_para.loc[(results_new_para['new_para_cor_p_baseline']<0.05) & 
(results_new_para['new_para_mv_age_p_baseline'] >= 0.05),
['var1', 'var2', 'new_para_cor_p_baseline', 'new_para_cor_r_baseline', 'new_para_cor_n', 'new_para_mv_age_p_baseline', 
'new_para_mv_age_B_baseline', 'new_para_mv_age_n', 'type']].reset_index(drop = True)

,var1,var2,new_para_cor_p_baseline,new_para_cor_r_baseline,new_para_cor_n,new_para_mv_age_p_baseline,new_para_mv_age_B_baseline,new_para_mv_age_n,type
0,Ab42_40,Par_gm,0.040975,0.342342,36,0.128642,0.262071,35,FDG PET
1,Ab42_40,Temp_gm,0.045535,0.335393,36,0.111153,0.268312,35,FDG PET
2,pTau217,SensMot_gm,0.038304,-0.337401,38,0.053055,-0.316313,37,FDG PET
3,pTau217,Precun_gm,0.013190,-0.398577,38,0.079768,-0.289403,37,FDG PET
4,Ab40,pTau181,0.046590,-0.308833,42,0.141322,-0.249397,39,Plasma
5,Ab42,NFL,0.022278,0.369870,38,0.068675,0.310093,35,Plasma
6,Ab42,pTau231,0.047037,-0.333226,36,0.099934,-0.289168,33,Plasma
7,NFL,Ab42,0.022278,0.369870,38,0.068675,0.321993,35,Plasma
8,NFL,pTau217,0.023581,0.361901,39,0.295995,0.195039,37,Plasma
9,pTau181,Ab40,0.046590,-0.308833,42,0.141322,-0.237027,39,Plasma


In [78]:
# significance gained by adding covariates
results_new_para.loc[(results_new_para['new_para_cor_p_baseline']>=0.05) & 
(results_new_para['new_para_mv_age_p_baseline'] < 0.05),
['var1', 'var2', 'new_para_cor_p_baseline', 'new_para_cor_r_baseline', 'new_para_cor_n', 'new_para_mv_age_p_baseline', 
'new_para_mv_age_B_baseline', 'new_para_mv_age_n', 'type']].reset_index(drop = True)

,var1,var2,new_para_cor_p_baseline,new_para_cor_r_baseline,new_para_cor_n,new_para_mv_age_p_baseline,new_para_mv_age_B_baseline,new_para_mv_age_n,type
0,Ab42_40,Precun_gm,0.175968,0.230631,36,0.026605,0.376949,35,FDG PET
1,GFAP,FRONTAL_gm,0.063296,-0.289129,42,0.038012,-0.331277,41,FDG PET
2,GFAP,Precun_gm,0.182021,-0.209959,42,0.042803,-0.328679,41,FDG PET
3,Ab42_40,pTau181,0.164136,0.230344,38,0.004839,0.469415,35,Plasma
4,Ab42_40,pTau231,0.093330,0.283912,36,0.001587,0.538329,33,Plasma
5,Ab42_40,pTau181_Ab42,0.132487,0.248495,38,0.011804,0.428299,35,Plasma
6,NFL,pTau181,0.105834,0.247119,44,0.036299,0.330747,41,Plasma
7,pTau181,Ab42_40,0.164136,0.230344,38,0.004839,0.474380,35,Plasma
8,pTau181,NFL,0.105834,0.247119,44,0.036299,0.333445,41,Plasma
9,pTau231,Ab42_40,0.093330,0.283912,36,0.001587,0.532646,33,Plasma


In [79]:
# difference in p-values
(results_new_para['new_para_cor_p_baseline'] - results_new_para['new_para_mv_age_p_baseline'] ).describe().round(3)

count    954.000
mean       0.025
std        0.209
min       -0.742
25%       -0.083
50%        0.006
75%        0.143
max        0.751
dtype: float64

## age vs age and education

In [80]:
# significance lost by adding covariates
results_new_para.loc[(results_new_para['new_para_mv_age_p_baseline']<0.05) & 
(results_new_para['new_para_mv_ageedu_p_baseline'] >= 0.05),
['var1', 'var2', 'new_para_mv_age_p_baseline',  'new_para_mv_age_B_baseline', 'new_para_mv_age_n',
       'new_para_mv_ageedu_p_baseline', 'new_para_mv_ageedu_B_baseline',  'new_para_mv_ageedu_n', 'type']].reset_index(drop = True)

,var1,var2,new_para_mv_age_p_baseline,new_para_mv_age_B_baseline,new_para_mv_age_n,new_para_mv_ageedu_p_baseline,new_para_mv_ageedu_B_baseline,new_para_mv_ageedu_n,type
0,GFAP,FRONTAL_gm,0.038012,-0.331277,41,0.116266,-0.263333,41,FDG PET
1,GFAP,Precun_gm,0.042803,-0.328679,41,0.141319,-0.254570,41,FDG PET
2,GFAP,RASref1_gm,0.018361,-0.394031,41,0.065472,-0.326914,41,FDG PET
3,pTau181,tma,0.041674,0.378808,34,0.056081,0.345504,34,Cognitive
4,GFAP,VOL_Hip_Rz,0.046973,0.339147,36,0.053214,0.326464,36,Volumetric
5,pTau217,VOL_Ventricles_Rz,0.049041,0.347275,32,0.054523,0.336066,32,Volumetric
6,pTau217,VOL_Parietal_Lz,0.045699,-0.412356,32,0.066935,-0.379981,32,Volumetric


In [81]:
# significance gained by adding covariates
results_new_para.loc[(results_new_para['new_para_mv_age_p_baseline']>=0.05) & 
(results_new_para['new_para_mv_ageedu_p_baseline'] < 0.05),
['var1', 'var2', 'new_para_mv_age_p_baseline',   'new_para_mv_age_B_baseline', 'new_para_mv_age_n',
       'new_para_mv_ageedu_p_baseline', 'new_para_mv_ageedu_B_baseline',  'new_para_mv_ageedu_n', 'type']].reset_index(drop = True)

,var1,var2,new_para_mv_age_p_baseline,new_para_mv_age_B_baseline,new_para_mv_age_n,new_para_mv_ageedu_p_baseline,new_para_mv_ageedu_B_baseline,new_para_mv_ageedu_n,type
0,pTau181,FRONTAL_gm,0.199865,0.204362,41,0.031049,0.348622,41,FDG PET
1,pTau181,PostCing_gm,0.052931,0.304133,41,0.004673,0.441786,41,FDG PET
2,pTau231,PostCing_gm,0.066360,0.311201,37,0.031009,0.385201,37,FDG PET
3,pTau181_Ab42,MTL_gm,0.199829,0.222713,35,0.008643,0.467339,35,FDG PET
4,NFL,tmb,0.064523,0.431735,19,0.048453,0.477913,19,Cognitive
...,...,...,...,...,...,...,...,...,...
60,pTau181_Ab42,VOL_Precun_InfPar_Supramarg_Rz,0.108750,0.311451,30,0.001254,0.563194,30,Volumetric
61,pTau181_Ab42,VOL_LatOccLingCun_Lz,0.071093,0.331401,30,0.026065,0.362568,30,Volumetric
62,pTau181_Ab42,VOL_LatOccLingCun_Rz,0.075020,0.327445,30,0.009236,0.423286,30,Volumetric
63,pTau181_Ab42,VOL_InfParSupra_Lz,0.144645,0.306248,30,0.036780,0.390568,30,Volumetric


In [106]:
# significance gained by adding covariates
results_new_para.loc[(results_new_para['new_para_mv_age_p_baseline']>=0.05) & 
(results_new_para['new_para_mv_ageedu_p_baseline'] < 0.05),
[ 'type']].value_counts()

type      
Volumetric    59
FDG PET        4
Cognitive      2
Name: count, dtype: int64

In [82]:
# difference in p-values
(results_new_para['new_para_mv_age_p_baseline'] - results_new_para['new_para_mv_ageedu_p_baseline'] ).describe().round(3)

count    954.000
mean       0.022
std        0.126
min       -0.473
25%       -0.031
50%        0.007
75%        0.067
max        0.588
dtype: float64

## age and education vs all 5 (age, education, apoe, sex, race)

In [83]:
# significance lost by adding covariates
results_new_para.loc[(results_new_para['new_para_mv_ageedu_p_baseline']<0.05) & 
(results_new_para['new_para_mv_aeasr_p_baseline'] >= 0.05),
['var1', 'var2', 'new_para_mv_ageedu_p_baseline', 'new_para_mv_ageedu_B_baseline',
       'new_para_mv_ageedu_n', 'new_para_mv_aeasr_p_baseline',
       'new_para_mv_aeasr_B_baseline', 'new_para_mv_aeasr_n', 'type']].reset_index(drop = True)

,var1,var2,new_para_mv_ageedu_p_baseline,new_para_mv_ageedu_B_baseline,new_para_mv_ageedu_n,new_para_mv_aeasr_p_baseline,new_para_mv_aeasr_B_baseline,new_para_mv_aeasr_n,type
0,Ab42_40,MTL_gm,0.016841,0.459326,35,0.094494,0.452294,30,FDG PET
1,pTau181,FRONTAL_gm,0.031049,0.348622,41,0.094632,0.317103,36,FDG PET
2,pTau231,MTL_gm,0.014014,0.453113,37,0.071912,0.480989,32,FDG PET
3,pTau181_Ab42,MTL_gm,0.008643,0.467339,35,0.061639,0.494528,30,FDG PET
4,GFAP,pTau217,0.026726,0.408415,37,0.115845,0.317162,33,Plasma
5,pTau217,GFAP,0.026726,0.343122,37,0.115845,0.291276,33,Plasma
6,NFL,tmb,0.048453,0.477913,19,0.117391,0.453023,19,Cognitive
7,Ab40,VOL_MidTemp_Lz,0.033639,-0.419898,34,0.074042,-0.452477,29,Volumetric
8,Ab40,VOL_InfPar_Lz,0.038276,-0.422644,34,0.087391,-0.399884,29,Volumetric
9,Ab40,VOL_Inf_Mid_Fus_Temp_Lz,0.039202,-0.386115,34,0.083807,-0.388404,29,Volumetric


In [84]:
# significance gained by adding covariates
results_new_para.loc[(results_new_para['new_para_mv_ageedu_p_baseline']>=0.05) & 
(results_new_para['new_para_mv_aeasr_p_baseline'] < 0.05),
['var1', 'var2', 'new_para_mv_ageedu_p_baseline', 'new_para_mv_ageedu_B_baseline',
       'new_para_mv_ageedu_n', 'new_para_mv_aeasr_p_baseline',
       'new_para_mv_aeasr_B_baseline', 'new_para_mv_aeasr_n', 'type']].reset_index(drop = True)

,var1,var2,new_para_mv_ageedu_p_baseline,new_para_mv_ageedu_B_baseline,new_para_mv_ageedu_n,new_para_mv_aeasr_p_baseline,new_para_mv_aeasr_B_baseline,new_para_mv_aeasr_n,type
0,Ab40,SensMot_gm,0.076043,0.295868,39,0.025934,0.441393,34,FDG PET
1,pTau217,SensMot_gm,0.050653,-0.320993,37,0.024095,-0.476497,33,FDG PET
2,pTau217,cowattotal,0.108647,-0.296203,34,0.035282,-0.430309,33,Cognitive
3,Ab42,VOL_InfTemp_Lz,0.079282,-0.332517,30,0.023488,-0.529657,25,Volumetric
4,Ab42,VOL_MidTemp_Rz,0.161545,-0.269894,30,0.015840,-0.749644,25,Volumetric
5,Ab42,VOL_InfPar_Rz,0.071140,-0.389029,30,0.008457,-0.776933,25,Volumetric
6,Ab42,VOL_InfFront_Lz,0.069335,-0.346041,30,0.011333,-0.581402,25,Volumetric
7,Ab42,VOL_MidFront_Lz,0.270771,-0.240442,30,0.033886,-0.582940,25,Volumetric
8,Ab42,VOL_SupFront_Rz,0.251826,-0.227472,30,0.047335,-0.520237,25,Volumetric
9,Ab42,VOL_Pericalc_Rz,0.073484,0.330155,30,0.037794,0.469488,25,Volumetric


In [85]:
# difference in p-values
(results_new_para['new_para_mv_ageedu_p_baseline'] - results_new_para['new_para_mv_aeasr_p_baseline'] ).describe().round(3)

count    954.000
mean      -0.006
std        0.167
min       -0.621
25%       -0.084
50%       -0.000
75%        0.065
max        0.685
dtype: float64

## age vs all 5 (age, education, apoe, sex, race)

In [86]:
# significance lost by adding covariates
results_new_para.loc[(results_new_para['new_para_mv_age_p_baseline']<0.05) & 
(results_new_para['new_para_mv_aeasr_p_baseline'] >= 0.05),
['var1', 'var2', 'new_para_mv_age_p_baseline', 'new_para_mv_ageedu_B_baseline',
       'new_para_mv_age_n', 'new_para_mv_aeasr_p_baseline',
       'new_para_mv_aeasr_B_baseline', 'new_para_mv_aeasr_n', 'type']].reset_index(drop= True)

,var1,var2,new_para_mv_age_p_baseline,new_para_mv_ageedu_B_baseline,new_para_mv_age_n,new_para_mv_aeasr_p_baseline,new_para_mv_aeasr_B_baseline,new_para_mv_aeasr_n,type
0,Ab42_40,MTL_gm,0.026321,0.459326,35,0.094494,0.452294,30,FDG PET
1,GFAP,FRONTAL_gm,0.038012,-0.263333,41,0.051380,-0.361989,36,FDG PET
2,GFAP,Precun_gm,0.042803,-0.254570,41,0.115938,-0.297232,36,FDG PET
3,GFAP,RASref1_gm,0.018361,-0.326914,41,0.102489,-0.417824,36,FDG PET
4,pTau231,MTL_gm,0.042440,0.453113,37,0.071912,0.480989,32,FDG PET
5,GFAP,pTau217,0.018361,0.408415,37,0.115845,0.317162,33,Plasma
6,pTau217,GFAP,0.018361,0.343122,37,0.115845,0.291276,33,Plasma
7,pTau181,tma,0.041674,0.345504,34,0.060669,0.372145,33,Cognitive
8,GFAP,VOL_InfParSupra_Lz,0.038333,-0.397833,36,0.057829,-0.410325,31,Volumetric
9,pTau217,VOL_Ventricles_Rz,0.049041,0.336066,32,0.160988,0.295956,28,Volumetric


In [87]:
# significance gained by adding covariates
results_new_para.loc[(results_new_para['new_para_mv_age_p_baseline']>=0.05) & 
(results_new_para['new_para_mv_aeasr_p_baseline'] < 0.05),
['var1', 'var2', 'new_para_mv_age_p_baseline', 'new_para_mv_ageedu_B_baseline',
       'new_para_mv_age_n', 'new_para_mv_aeasr_p_baseline',
       'new_para_mv_aeasr_B_baseline', 'new_para_mv_aeasr_n', 'type']].reset_index(drop= True)

,var1,var2,new_para_mv_age_p_baseline,new_para_mv_ageedu_B_baseline,new_para_mv_age_n,new_para_mv_aeasr_p_baseline,new_para_mv_aeasr_B_baseline,new_para_mv_aeasr_n,type
0,Ab40,SensMot_gm,0.068632,0.295868,39,0.025934,0.441393,34,FDG PET
1,pTau181,PostCing_gm,0.052931,0.441786,41,0.002768,0.513738,36,FDG PET
2,pTau217,SensMot_gm,0.053055,-0.320993,37,0.024095,-0.476497,33,FDG PET
3,pTau231,PostCing_gm,0.066360,0.385201,37,0.011149,0.509237,32,FDG PET
4,pTau181,adltotal,0.059993,0.373911,37,0.031447,0.379565,36,Cognitive
...,...,...,...,...,...,...,...,...,...
72,pTau181_Ab42,VOL_Precun_InfPar_Supramarg_Rz,0.108750,0.563194,30,0.003669,0.630301,25,Volumetric
73,pTau181_Ab42,VOL_LatOccLingCun_Lz,0.071093,0.362568,30,0.007344,0.512186,25,Volumetric
74,pTau181_Ab42,VOL_LatOccLingCun_Rz,0.075020,0.423286,30,0.009140,0.503818,25,Volumetric
75,pTau181_Ab42,VOL_InfParSupra_Lz,0.144645,0.390568,30,0.037092,0.463150,25,Volumetric


In [88]:
# difference in p-values
(results_new_para['new_para_mv_age_p_baseline'] - results_new_para['new_para_mv_aeasr_p_baseline'] ).describe().round(3)

count    954.000
mean       0.016
std        0.194
min       -0.742
25%       -0.077
50%        0.009
75%        0.112
max        0.802
dtype: float64

# New Para with and without RIL8

In [89]:
comp_new_para = pd.merge(results_orig_new_para, results_new_para, how = 'inner')
comp_new_para

,var1,var2,orig_new_para_cor_p_baseline,orig_new_para_cor_r_baseline,orig_new_para_cor_n,orig_new_para_mv_age_p_baseline,orig_new_para_mv_age_B_baseline,orig_new_para_mv_age_n,orig_new_para_mv_ageedu_p_baseline,orig_new_para_mv_ageedu_B_baseline,...,new_para_cor_n,new_para_mv_age_p_baseline,new_para_mv_age_B_baseline,new_para_mv_age_n,new_para_mv_ageedu_p_baseline,new_para_mv_ageedu_B_baseline,new_para_mv_ageedu_n,new_para_mv_aeasr_p_baseline,new_para_mv_aeasr_B_baseline,new_para_mv_aeasr_n
0,Ab40,MTL_gm,0.272100,-0.175610,41,0.879090,-0.025949,40,0.694780,-0.077208,...,40,0.888369,-0.024268,39,0.682126,-0.081689,39,0.998305,0.000562,34
1,Ab40,SensMot_gm,0.054759,0.302265,41,0.063042,0.301542,40,0.069319,0.298690,...,40,0.068632,0.299515,39,0.076043,0.295868,39,0.025934,0.441393,34
2,Ab40,FRONTAL_gm,0.997412,0.000523,41,0.435330,0.128753,40,0.500755,0.119801,...,40,0.480051,0.118418,39,0.568897,0.104043,39,0.529761,0.139394,34
3,Ab40,AC_gm,0.527467,-0.101568,41,0.826350,-0.037865,40,0.749702,-0.057259,...,40,0.768005,-0.051412,39,0.672515,-0.077033,39,0.941134,0.017332,34
4,Ab40,Precun_gm,0.640798,0.075087,41,0.857469,0.030100,40,0.972479,0.006278,...,40,0.940730,0.012692,39,0.905003,-0.022502,39,0.899370,-0.027883,34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
958,pTau181_Ab42,VOL_Precun_InfPar_Supramarg_Rz,0.456771,0.138710,31,0.104883,0.313279,31,0.003827,0.535494,...,30,0.108750,0.311451,30,0.001254,0.563194,30,0.003669,0.630301,25
959,pTau181_Ab42,VOL_LatOccLingCun_Lz,0.101066,0.300000,31,0.036124,0.378357,31,0.012904,0.415780,...,30,0.071093,0.331401,30,0.026065,0.362568,30,0.007344,0.512186,25
960,pTau181_Ab42,VOL_LatOccLingCun_Rz,0.062338,0.338710,31,0.040792,0.370085,31,0.005581,0.464798,...,30,0.075020,0.327445,30,0.009236,0.423286,30,0.009140,0.503818,25
961,pTau181_Ab42,VOL_InfParSupra_Lz,0.601511,0.097581,31,0.218825,0.257036,31,0.100800,0.321389,...,30,0.144645,0.306248,30,0.036780,0.390568,30,0.037092,0.463150,25


## cor

In [90]:
comp_new_para[(comp_new_para['orig_new_para_cor_p_baseline'] < 0.05) & (comp_new_para['new_para_cor_p_baseline'] >= 0.05)]

,var1,var2,orig_new_para_cor_p_baseline,orig_new_para_cor_r_baseline,orig_new_para_cor_n,orig_new_para_mv_age_p_baseline,orig_new_para_mv_age_B_baseline,orig_new_para_mv_age_n,orig_new_para_mv_ageedu_p_baseline,orig_new_para_mv_ageedu_B_baseline,...,new_para_cor_n,new_para_mv_age_p_baseline,new_para_mv_age_B_baseline,new_para_mv_age_n,new_para_mv_ageedu_p_baseline,new_para_mv_ageedu_B_baseline,new_para_mv_ageedu_n,new_para_mv_aeasr_p_baseline,new_para_mv_aeasr_B_baseline,new_para_mv_aeasr_n
14,Ab42,Precun_gm,0.043939,0.333116,37,0.067379,0.305760,36,0.117898,0.278135,...,36,0.090004,0.290714,35,0.170057,0.254287,35,0.388394,0.192637,30
32,GFAP,FRONTAL_gm,0.033776,-0.324461,43,0.022479,-0.356626,42,0.054874,-0.316305,...,42,0.038012,-0.331277,41,0.116266,-0.263333,41,0.051380,-0.361989,36
62,pTau217,FRONTAL_gm,0.049762,-0.316344,39,0.050892,-0.317208,38,0.058460,-0.322643,...,38,0.093915,-0.274025,37,0.136321,-0.256450,37,0.075504,-0.362132,33
118,GFAP,pTau181,0.028186,0.327305,45,0.045342,0.314916,42,0.084886,0.276985,...,44,0.156823,0.229949,41,0.336088,0.159868,41,0.266473,0.201123,36
133,pTau181,GFAP,0.028186,0.327305,45,0.045342,0.313735,42,0.084886,0.274751,...,44,0.156823,0.226332,41,0.336088,0.156560,41,0.266473,0.210779,36
226,pTau181,tma,0.023398,0.382367,35,0.025987,0.401538,35,0.032228,0.383712,...,34,0.041674,0.378808,34,0.056081,0.345504,34,0.060669,0.372145,33
359,Ab42,VOL_SupTemp_Lz,0.044396,-0.363545,31,0.024700,-0.402400,31,0.026893,-0.397646,...,30,0.027061,-0.402231,30,0.029308,-0.396463,30,0.002687,-0.617715,25
403,Ab42,VOL_LatTemp_Rz,0.041557,-0.368182,31,0.037446,-0.375016,31,0.013817,-0.453929,...,30,0.045267,-0.367921,30,0.018458,-0.438603,30,0.005060,-0.645116,25
414,Ab42,VOL_Inf_Mid_Fus_Temp_Lz,0.043392,-0.365158,31,0.010995,-0.461025,31,0.002734,-0.553353,...,30,0.014190,-0.449607,30,0.004061,-0.531441,30,0.000870,-0.707149,25
533,GFAP,VOL_SupFront_Lz,0.024753,-0.368658,37,0.020790,-0.389889,37,0.016937,-0.401601,...,36,0.052831,-0.337304,36,0.044574,-0.344360,36,0.034493,-0.413301,31


In [91]:
comp_new_para[(comp_new_para['orig_new_para_cor_p_baseline'] >= 0.05) & (comp_new_para['new_para_cor_p_baseline'] < 0.05)]

,var1,var2,orig_new_para_cor_p_baseline,orig_new_para_cor_r_baseline,orig_new_para_cor_n,orig_new_para_mv_age_p_baseline,orig_new_para_mv_age_B_baseline,orig_new_para_mv_age_n,orig_new_para_mv_ageedu_p_baseline,orig_new_para_mv_ageedu_B_baseline,...,new_para_cor_n,new_para_mv_age_p_baseline,new_para_mv_age_B_baseline,new_para_mv_age_n,new_para_mv_ageedu_p_baseline,new_para_mv_ageedu_B_baseline,new_para_mv_ageedu_n,new_para_mv_aeasr_p_baseline,new_para_mv_aeasr_B_baseline,new_para_mv_aeasr_n
203,GFAP,tmb,0.095694,0.382851,20,0.065837,0.386686,20,0.084706,0.373753,...,19,0.016344,0.511666,19,0.019316,0.525796,19,0.000480,0.729965,19
204,GFAP,cowattotal,0.090952,-0.278070,38,0.225670,-0.202904,38,0.276696,-0.190613,...,37,0.110717,-0.269607,37,0.163898,-0.243575,37,0.053655,-0.373456,36
237,pTau217,dstotal,0.052345,-0.330685,35,0.216293,-0.223839,35,0.184718,-0.257554,...,34,0.068688,-0.325775,34,0.077928,-0.337331,34,0.056728,-0.397761,33
851,pTau231,VOL_Pericalc_Lz,0.087832,-0.301805,33,0.032964,-0.377102,33,0.044985,-0.359620,...,32,0.005266,-0.486535,32,0.007527,-0.467931,32,0.018624,-0.498641,27
875,pTau231,VOL_MidSupFrontal_Rz,0.068063,0.321524,33,0.063350,0.346055,33,0.058018,0.353230,...,32,0.045846,0.376036,32,0.038608,0.386068,32,0.020197,0.569503,27
907,pTau181_Ab42,VOL_SupraMarg_Rz,0.056285,0.346371,31,0.031539,0.388137,31,0.033724,0.360827,...,30,0.023143,0.410493,30,0.020433,0.377231,30,0.012915,0.558191,25
915,pTau181_Ab42,VOL_InfFront_Rz,0.101066,0.300000,31,0.078665,0.324046,31,0.064905,0.318201,...,30,0.044292,0.369474,30,0.025259,0.366424,30,0.024506,0.420692,25


In [92]:
(comp_new_para['orig_new_para_cor_p_baseline']  - comp_new_para['new_para_cor_p_baseline']).describe()

count    954.000000
mean      -0.013384
std        0.118107
min       -0.357496
25%       -0.065380
50%       -0.011042
75%        0.035815
max        0.394858
dtype: float64

## age

In [93]:
comp_new_para.loc[(comp_new_para['orig_new_para_mv_age_p_baseline'] < 0.05) & (comp_new_para['new_para_mv_age_p_baseline'] >= 0.05), ['var1', 'var2', 'orig_new_para_mv_age_p_baseline',
       'orig_new_para_mv_age_B_baseline', 'orig_new_para_mv_age_n', 'new_para_mv_age_p_baseline', 'new_para_mv_age_B_baseline',
       'new_para_mv_age_n']]

,var1,var2,orig_new_para_mv_age_p_baseline,orig_new_para_mv_age_B_baseline,orig_new_para_mv_age_n,new_para_mv_age_p_baseline,new_para_mv_age_B_baseline,new_para_mv_age_n
35,GFAP,Par_gm,0.031328,-0.351754,42,0.073024,-0.302208,41
64,pTau217,Precun_gm,0.028535,-0.356517,38,0.079768,-0.289403,37
118,GFAP,pTau181,0.045342,0.314916,42,0.156823,0.229949,41
133,pTau181,GFAP,0.045342,0.313735,42,0.156823,0.226332,41
144,pTau217,pTau231,0.029094,0.359666,35,0.157231,0.236624,34
152,pTau231,pTau217,0.029094,0.389976,35,0.157231,0.268259,34
198,GFAP,MMSE,0.035420,-0.341870,38,0.057730,-0.316496,37
231,pTau181,adltotal,0.037642,0.343085,38,0.059993,0.315294,37
262,pTau181_Ab42,tma,0.047857,0.393763,30,0.079848,0.360600,29
286,Ab40,VOL_InfPar_Lz,0.045246,-0.393358,35,0.052674,-0.385510,34


In [94]:
comp_new_para[(comp_new_para['orig_new_para_mv_age_p_baseline'] >= 0.05) & (comp_new_para['new_para_mv_age_p_baseline'] < 0.05)]

,var1,var2,orig_new_para_cor_p_baseline,orig_new_para_cor_r_baseline,orig_new_para_cor_n,orig_new_para_mv_age_p_baseline,orig_new_para_mv_age_B_baseline,orig_new_para_mv_age_n,orig_new_para_mv_ageedu_p_baseline,orig_new_para_mv_ageedu_B_baseline,...,new_para_cor_n,new_para_mv_age_p_baseline,new_para_mv_age_B_baseline,new_para_mv_age_n,new_para_mv_ageedu_p_baseline,new_para_mv_ageedu_B_baseline,new_para_mv_ageedu_n,new_para_mv_aeasr_p_baseline,new_para_mv_aeasr_B_baseline,new_para_mv_aeasr_n
203,GFAP,tmb,0.095694,0.382851,20,0.065837,0.386686,20,0.084706,0.373753,...,19,0.016344,0.511666,19,0.019316,0.525796,19,0.000480,0.729965,19
389,Ab42,VOL_Pericalc_Lz,0.200583,0.236314,31,0.063188,0.341828,31,0.086460,0.321796,...,30,0.028602,0.401856,30,0.039210,0.383081,30,0.016598,0.522512,25
733,pTau217,VOL_Ventricles_Rz,0.218671,0.219975,33,0.211248,0.227715,33,0.230586,0.221316,...,32,0.049041,0.347275,32,0.054523,0.336066,32,0.160988,0.295956,28
748,pTau217,VOL_InfPar_Lz,0.018786,-0.406853,33,0.111679,-0.328960,33,0.138042,-0.315707,...,32,0.018954,-0.466387,32,0.030432,-0.434097,32,0.059733,-0.461059,28
789,pTau217,VOL_Parietal_Lz,0.012828,-0.428583,33,0.054821,-0.400099,33,0.067801,-0.389143,...,32,0.045699,-0.412356,32,0.066935,-0.379981,32,0.082298,-0.424370,28
801,pTau217,VOL_Precun_InfPar_Lz,0.020851,-0.400669,33,0.070316,-0.377409,33,0.088920,-0.365382,...,32,0.028436,-0.445925,32,0.046267,-0.410420,32,0.075410,-0.438172,28
875,pTau231,VOL_MidSupFrontal_Rz,0.068063,0.321524,33,0.063350,0.346055,33,0.058018,0.353230,...,32,0.045846,0.376036,32,0.038608,0.386068,32,0.020197,0.569503,27
910,pTau181_Ab42,VOL_OrbitFront_Lz,0.101548,0.299597,31,0.051910,0.353764,31,0.091230,0.295396,...,30,0.028475,0.397345,30,0.048904,0.328971,30,0.019087,0.459929,25
915,pTau181_Ab42,VOL_InfFront_Rz,0.101066,0.300000,31,0.078665,0.324046,31,0.064905,0.318201,...,30,0.044292,0.369474,30,0.025259,0.366424,30,0.024506,0.420692,25


In [95]:
(comp_new_para['orig_new_para_mv_age_p_baseline']  - comp_new_para['new_para_mv_age_p_baseline'])[(comp_new_para['orig_new_para_mv_age_p_baseline']  - comp_new_para['new_para_mv_age_p_baseline']) != 0].describe().round(3).round(2)

count    954.00
mean      -0.02
std        0.15
min       -0.66
25%       -0.05
50%       -0.00
75%        0.02
max        0.78
dtype: float64

In [96]:
(comp_new_para['orig_new_para_mv_age_p_baseline']  - comp_new_para['new_para_mv_age_p_baseline']).describe().round(3)

count    954.000
mean      -0.016
std        0.149
min       -0.655
25%       -0.050
50%       -0.005
75%        0.022
max        0.785
dtype: float64

In [97]:
(comp_new_para['orig_new_para_mv_age_n']  - comp_new_para['new_para_mv_age_n']).value_counts()

1    963
Name: count, dtype: int64

## age education

In [98]:
comp_new_para.loc[(comp_new_para['orig_new_para_mv_ageedu_p_baseline'] < 0.05) & (comp_new_para['new_para_mv_ageedu_p_baseline'] >= 0.05), ['var1', 'var2', 'orig_new_para_mv_ageedu_p_baseline',
       'orig_new_para_mv_ageedu_B_baseline', 'orig_new_para_mv_ageedu_n', 'new_para_mv_ageedu_p_baseline', 'new_para_mv_ageedu_B_baseline',
       'new_para_mv_ageedu_n']]

,var1,var2,orig_new_para_mv_ageedu_p_baseline,orig_new_para_mv_ageedu_B_baseline,orig_new_para_mv_ageedu_n,new_para_mv_ageedu_p_baseline,new_para_mv_ageedu_B_baseline,new_para_mv_ageedu_n
34,GFAP,Precun_gm,0.047406,-0.332893,42,0.141319,-0.254570,41
35,GFAP,Par_gm,0.049568,-0.322200,42,0.130108,-0.254096,41
64,pTau217,Precun_gm,0.031974,-0.370614,38,0.120831,-0.274519,37
144,pTau217,pTau231,0.032806,0.363630,35,0.203215,0.221750,34
152,pTau231,pTau217,0.032806,0.381457,35,0.203215,0.240789,34
198,GFAP,MMSE,0.046081,-0.343655,38,0.098455,-0.293590,37
226,pTau181,tma,0.032228,0.383712,35,0.056081,0.345504,34
297,Ab40,VOL_Insula_Rz,0.048031,-0.344385,35,0.060585,-0.334067,34
331,Ab40,VOL_InfMidTemp_Lz,0.047146,-0.379460,35,0.056812,-0.366006,34
400,Ab42,VOL_TotalGray_Lz,0.045473,-0.399686,31,0.053449,-0.389353,30


In [99]:
comp_new_para.loc[(comp_new_para['orig_new_para_mv_ageedu_p_baseline'] >= 0.05) & (comp_new_para['new_para_mv_ageedu_p_baseline'] < 0.05), ['var1', 'var2', 'orig_new_para_mv_ageedu_p_baseline',
       'orig_new_para_mv_ageedu_B_baseline', 'orig_new_para_mv_ageedu_n', 'new_para_mv_ageedu_p_baseline', 'new_para_mv_ageedu_B_baseline',
       'new_para_mv_ageedu_n']]

,var1,var2,orig_new_para_mv_ageedu_p_baseline,orig_new_para_mv_ageedu_B_baseline,orig_new_para_mv_ageedu_n,new_para_mv_ageedu_p_baseline,new_para_mv_ageedu_B_baseline,new_para_mv_ageedu_n
52,pTau181,FRONTAL_gm,0.132302,0.246818,42,0.031049,0.348622,41
59,pTau181,PostCing_gm,0.099669,0.264216,42,0.004673,0.441786,41
79,pTau231,PostCing_gm,0.198694,0.227930,38,0.031009,0.385201,37
203,GFAP,tmb,0.084706,0.373753,20,0.019316,0.525796,19
215,NFL,tmb,0.052504,0.452187,20,0.048453,0.477913,19
389,Ab42,VOL_Pericalc_Lz,0.086460,0.321796,31,0.039210,0.383081,30
422,Ab42,VOL_InfParSupra_Lz,0.057208,-0.393428,31,0.047271,-0.415674,30
498,Ab42_40,VOL_LatOccLingCun_Rz,0.053761,0.357243,31,0.047735,0.372335,30
748,pTau217,VOL_InfPar_Lz,0.138042,-0.315707,33,0.030432,-0.434097,32
801,pTau217,VOL_Precun_InfPar_Lz,0.088920,-0.365382,33,0.046267,-0.410420,32


In [100]:
(comp_new_para['orig_new_para_mv_ageedu_p_baseline']  - comp_new_para['new_para_mv_ageedu_p_baseline']).describe().round(3)

count    954.000
mean      -0.015
std        0.158
min       -0.864
25%       -0.048
50%       -0.004
75%        0.027
max        0.756
dtype: float64

In [101]:
comp_new_para.loc[(abs(comp_new_para['orig_new_para_mv_ageedu_p_baseline']  - comp_new_para['new_para_mv_ageedu_p_baseline'])> 0.1), ['var1', 'var2', 'orig_new_para_mv_ageedu_p_baseline',
       'orig_new_para_mv_ageedu_B_baseline', 'orig_new_para_mv_ageedu_n', 'new_para_mv_ageedu_p_baseline', 'new_para_mv_ageedu_B_baseline',
       'new_para_mv_ageedu_n']].sort_values('new_para_mv_ageedu_p_baseline')

,var1,var2,orig_new_para_mv_ageedu_p_baseline,orig_new_para_mv_ageedu_B_baseline,orig_new_para_mv_ageedu_n,new_para_mv_ageedu_p_baseline,new_para_mv_ageedu_B_baseline,new_para_mv_ageedu_n
748,pTau217,VOL_InfPar_Lz,0.138042,-0.315707,33,0.030432,-0.434097,32
79,pTau231,PostCing_gm,0.198694,0.227930,38,0.031009,0.385201,37
52,pTau181,FRONTAL_gm,0.132302,0.246818,42,0.031049,0.348622,41
909,pTau181_Ab42,VOL_SupPar_Rz,0.286931,0.207182,31,0.048215,0.369900,30
733,pTau217,VOL_Ventricles_Rz,0.230586,0.221316,33,0.054523,0.336066,32
...,...,...,...,...,...,...,...,...
13,Ab42,AC_gm,0.838614,0.035259,36,0.985600,0.003163,35
312,Ab40,VOL_Pericalc_Lz,0.880801,-0.028797,35,0.993763,-0.001511,34
207,GFAP,adltotal,0.805843,0.042542,38,0.994705,-0.001159,37
541,GFAP,VOL_Cuneus_Lz,0.862887,0.030815,37,0.996220,-0.000846,36


## age education apoe sex race

In [102]:
comp_new_para.columns

Index(['var1', 'var2', 'orig_new_para_cor_p_baseline',
       'orig_new_para_cor_r_baseline', 'orig_new_para_cor_n',
       'orig_new_para_mv_age_p_baseline', 'orig_new_para_mv_age_B_baseline',
       'orig_new_para_mv_age_n', 'orig_new_para_mv_ageedu_p_baseline',
       'orig_new_para_mv_ageedu_B_baseline', 'orig_new_para_mv_ageedu_n',
       'orig_new_para_mv_aeasr_p_baseline',
       'orig_new_para_mv_aeasr_B_baseline', 'orig_new_para_mv_aeasr_n', 'type',
       'new_para_cor_p_baseline', 'new_para_cor_r_baseline', 'new_para_cor_n',
       'new_para_mv_age_p_baseline', 'new_para_mv_age_B_baseline',
       'new_para_mv_age_n', 'new_para_mv_ageedu_p_baseline',
       'new_para_mv_ageedu_B_baseline', 'new_para_mv_ageedu_n',
       'new_para_mv_aeasr_p_baseline', 'new_para_mv_aeasr_B_baseline',
       'new_para_mv_aeasr_n'],
      dtype='object')

In [103]:
comp_new_para[(comp_new_para['orig_new_para_mv_aeasr_p_baseline'] < 0.05) & (comp_new_para['new_para_mv_aeasr_p_baseline'] >= 0.05)]

,var1,var2,orig_new_para_cor_p_baseline,orig_new_para_cor_r_baseline,orig_new_para_cor_n,orig_new_para_mv_age_p_baseline,orig_new_para_mv_age_B_baseline,orig_new_para_mv_age_n,orig_new_para_mv_ageedu_p_baseline,orig_new_para_mv_ageedu_B_baseline,...,new_para_cor_n,new_para_mv_age_p_baseline,new_para_mv_age_B_baseline,new_para_mv_age_n,new_para_mv_ageedu_p_baseline,new_para_mv_ageedu_B_baseline,new_para_mv_ageedu_n,new_para_mv_aeasr_p_baseline,new_para_mv_aeasr_B_baseline,new_para_mv_aeasr_n


In [104]:
comp_new_para[(comp_new_para['orig_new_para_mv_aeasr_p_baseline'] >= 0.05) & (comp_new_para['new_para_mv_aeasr_p_baseline'] < 0.05)]

,var1,var2,orig_new_para_cor_p_baseline,orig_new_para_cor_r_baseline,orig_new_para_cor_n,orig_new_para_mv_age_p_baseline,orig_new_para_mv_age_B_baseline,orig_new_para_mv_age_n,orig_new_para_mv_ageedu_p_baseline,orig_new_para_mv_ageedu_B_baseline,...,new_para_cor_n,new_para_mv_age_p_baseline,new_para_mv_age_B_baseline,new_para_mv_age_n,new_para_mv_ageedu_p_baseline,new_para_mv_ageedu_B_baseline,new_para_mv_ageedu_n,new_para_mv_aeasr_p_baseline,new_para_mv_aeasr_B_baseline,new_para_mv_aeasr_n


In [105]:
(comp_new_para['orig_new_para_mv_aeasr_p_baseline']  - comp_new_para['new_para_mv_aeasr_p_baseline']).describe().round(3)

count    954.0
mean      -0.0
std        0.0
min       -0.0
25%       -0.0
50%        0.0
75%        0.0
max        0.0
dtype: float64